# ParProx

This package aims to perform penalized logistic regression and Cox proportional hazard regression using the proximal gradient method. It supports $\ell_1$ penalty (lasso) and the group lasso penalty. The groups may overlap, as postulated by the latent group lasso ([Obozinski,Jacob, and Vert, 2011](https://arxiv.org/abs/1110.0413)). This package also supports computation on a single Nvidia GPU via the package [CUDA.jl](https://juliagpu.gitlab.io/CUDA.jl/). 

## Installation

### Julia

Download Julia from [https://julialang.org/downloads/](https://julialang.org/downloads/) according to your OS and hardware, and follow platform-specific instruction [here](https://julialang.org/downloads/platform/). 

You can also run Julia from the Jupyter Notebook after installing the package [IJulia](https://github.com/JuliaLang/IJulia.jl):

```julia
using Pkg
pkg"add IJulia"
```

### The Package

The `ParProx` package is installed by using the command:

```julia
using Pkg
pkg"add https://github.com/kose-y/ParProx.jl"
```

To install the other packages used in this tutoral, use the command:
```julia
pkg"add Printf Statistics CSV Mmap CodecZlib ROCAnalysis DataFrames"
```

For optional CUDA support, the following is also needed:
```julia
pkg"add CUDA Adapt"
```

It requires a CUDA-capable GPU with compute capability 5.0 (Maxwell) or higher, and an accompanying Nvidia driver with support for CUDA 10.1 or newer. CUDA is automatically downloaded before the first use, unless configured otherwise through an environment variable. See [this page](https://juliagpu.gitlab.io/CUDA.jl/installation/overview/) for further information. 

The following shows the version of Julia and system information.

In [1]:
versioninfo()

Julia Version 1.5.2
Commit 539f3ce943 (2020-09-23 23:17 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) Silver 4114 CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake-avx512)


## Example 1: Cox regression with nonoverlapping groups

In this example, we perform the Cox regression on somatic mutation count data of 
Glioblastoma patients. We use count of mutations in protein units as the covariates, and the group lasso penalty is applied, where the variables are grouped by the gene where the unit belongs to.

### Loading packages

In [2]:
using ParProx, Printf, Statistics # load the packages
using CSV, DataFrames, CodecZlib, Mmap # packages for data reading. GZip is used to read the gzipped text file.

### Reading data files

The file "somatic_sum_table_GBM.tsv.gz" contains the information the mutations and the count of mutations. Columns 1-9 contains the information of the mutations, and the tenth column and later contains the mutation count for each subject.

In [3]:
somatic = DataFrame!(CSV.File(transcode(GzipDecompressor, Mmap.mmap(
            ParProx.datadir("somatic_sum_table_GBM.tsv.gz")))));

For regular tab- or comma-delimited files, `DataFrame!(CSV.File("test.txt"))` should be enough. 

We print first six rows and five columns of the dataframe:

In [4]:
first(somatic[!, 1:5], 6)

,uniprot_accession,start_position,end_position,center_position,unit_name
,String,String,String,String,String
1,Q9H2S6,NA,NA,NA,NCU
2,O60762,28,199,113,Glycos_transf_2
3,Q8IZE3,26,245,135,Pkinase
4,Q8IZE3,NA,NA,NA,LU
5,Q8IZE3,NA,NA,NA,NCU
6,Q9NSG2,176,728,452,DUF4487


and next four columns are:

In [5]:
first(somatic[!, 6:9], 6)

,gene_name,gene_id,unit_label,sum
,String?,String,String,Int64
1,TNMD,ENSG00000000005,NCU,1
2,DPM1,ENSG00000000419,PIU,1
3,SCYL3,ENSG00000000457,PIU,1
4,SCYL3,ENSG00000000457,LU,2
5,SCYL3,ENSG00000000457,NCU,1
6,C1orf112,ENSG00000000460,PIU,1


The actual count data starts at column 10. 

In [6]:
first(somatic[!, 10:15], 6)

,TCGA-02-0003,TCGA-02-0033,TCGA-02-0047,TCGA-02-0055,TCGA-02-2466,TCGA-02-2470
,Int64,Int64,Int64,Int64,Int64,Int64
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,0,0,0,0,0,0


The file "primary_TCGA_CDR_GBM.tsv" contains the clinical information of the subjects.

In [7]:
# read data
cdr = CSV.read("primary_TCGA_CDR_GBM.tsv", DataFrame);

In [8]:
first(cdr, 6)

,V1,bcr_patient_barcode,type,age_at_initial_pathologic_diagnosis,gender
,Int64,String,String,Int64,String
1,2647,TCGA-02-0003,GBM,50,MALE
2,2664,TCGA-02-0033,GBM,54,MALE
3,2671,TCGA-02-0047,GBM,78,MALE
4,2676,TCGA-02-0055,GBM,62,FEMALE
5,2734,TCGA-02-2466,GBM,61,MALE
6,2735,TCGA-02-2470,GBM,57,MALE


We take the survival outcome and survival time from the clinical data:

In [9]:
survival_event = convert(Vector, cdr[!, :OS])
survival_time = convert(Vector, cdr[!, Symbol("OS.time")]);

We will also add age at inital pathologic diagnosis and gender as unpenalized covariates. We will code males to 1 and females to 0. 

In [10]:
# age at initial pathological diagnosis
age_at_diagnosis = cdr[!, :age_at_initial_pathologic_diagnosis]

# gender: 1: male, 0: not male
gender = map(x -> x == "MALE", cdr[!, :gender]) 

# we will adjust for these two covariates by adding them as nonpenalized variables. 
clinical = hcat(age_at_diagnosis, gender);

We take the predictors in a single matrix:

In [11]:
somatic_predictors = hcat(convert(Array{Float64}, transpose(convert(Array{Float64}, somatic[:, 10:end]))), clinical)

390×25309 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  50.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  54.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  78.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  62.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  61.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  57.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  43.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  53.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  64.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  81.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  84.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  67.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  63.0  0.0
 ⋮        

Unpenalized variables must be placed at the last columns of the matrix.

we normalize the input variables:

In [12]:
normalize(x) = (x .- mean(x; dims=1))./ std(x; dims=1)
somatic_predictors = normalize(somatic_predictors)

390×25309 Array{Float64,2}:
 -0.050637  -0.050637  -0.050637  …  -0.0879321  -0.748927    0.764111
 -0.050637  -0.050637  -0.050637     -0.0879321  -0.450504    0.764111
 -0.050637  -0.050637  -0.050637     -0.0879321   1.34003     0.764111
 -0.050637  -0.050637  -0.050637     -0.0879321   0.146342   -1.30536
 -0.050637  -0.050637  -0.050637     -0.0879321   0.0717363   0.764111
 -0.050637  -0.050637  -0.050637  …  -0.0879321  -0.226687    0.764111
 -0.050637  -0.050637  -0.050637     -0.0879321  -1.27117     0.764111
 -0.050637  -0.050637  -0.050637     -0.0879321  -0.52511     0.764111
 -0.050637  -0.050637  -0.050637     -0.0879321   0.295553    0.764111
 -0.050637  -0.050637  -0.050637     -0.0879321   1.56385    -1.30536
 -0.050637  -0.050637  -0.050637  …  -0.0879321   1.78767    -1.30536
 -0.050637  -0.050637  -0.050637     -0.0879321   0.519371    0.764111
 -0.050637  -0.050637  -0.050637     -0.0879321   0.220948   -1.30536
  ⋮                               ⋱                  

We are considering 390 subjects and 25309 variables (25307 somatic mutations, age at diagnosis, and gender). 

We check if the order of clinical data and the mutation data matches.

In [13]:
all(cdr[!, :bcr_patient_barcode] .== names(somatic)[10:end])

true

### Sorting by nonincreasing order of observed time

The samples should be sorted in nonincreasing order of observed survival time. 

In [14]:
sortorder = sortperm(survival_time; rev=true)
somatic_predictors = normalize(somatic_predictors[sortorder, :])
survival_event = survival_event[sortorder]
survival_time = survival_time[sortorder]

390-element Array{Int64,1}:
 3881
 3667
 2883
 2791
 2246
 2126
 1818
 1788
 1481
 1458
 1448
 1426
 1417
    ⋮
   15
   13
   12
    6
    6
    6
    5
    4
    4
    3
    3
    0

The following code creates a vector of String to be used as the name of each variable. We will use the first eight columns of `somatic` data frame to make a variable name.

In [15]:
somatic_variable_names = map(x -> "$(x[1])\t$(x[2])\t$(x[3])\t" * 
    "$(x[4])\t$(x[5])\t$(x[6])\t$(x[7])\t$(x[8])", eachrow(somatic))
somatic_variable_names = vcat(somatic_variable_names, map(x -> "$x\t\t\t\t\t\t\t", 
        ["age_at_diagnosis", "gender"]));

### Group definition

We figure out unique group names (`somatic_groups`), size of each group (`somatic_group_sizes`), and which group each variable belongs to (`somatic_grpidx`).

In [16]:
somatic_groups = unique(somatic[!, :gene_id])
somatic_group_sizes = map(somatic_groups) do x
    count(y -> y == x, somatic[!, :gene_id])
end
somatic_grpidx = map(somatic[!, :gene_id]) do x 
    findfirst(y -> y .== x, somatic_groups)
end;

`somatic_grpidx` gives the group index each variable belongs. Note that Julia uses 1-based indexing.

In [17]:
somatic_grpidx

25307-element Array{Int64,1}:
     1
     2
     3
     3
     3
     4
     5
     5
     5
     5
     5
     5
     5
     ⋮
 14247
 14248
 14249
 14250
 14251
 14252
 14253
 14253
 14253
 14253
 14253
 14254

### Cross validation

For the cross validation, we create a `COXUpdate` object specifying the update rules: maximum number of iterations, steps between intermediate evaluation, relative tolerance for stopping, and whether to print the intermediate output.

For cross validation in Cox regression, "C-index" is used as the metric (the higher, the better). Stratified k-fold cross validation is implemented (k=5 here), i.e., we ensure that the proportion of event and censored subject as equal as possible across the folds.  

In [18]:
using Random
Random.seed!(222);

We perform 5-fold corss validation using CPU only:

In [19]:
T = Float64
A = Array
U = ParProx.COXUpdate(; maxiter=10000, step=10, tol=5e-4, verbose=true)
lambdas = 10 .^ (range(-4, stop=-6, length=21)) # 21 values equally log-spaced in 10^-5 .. 10^-7
penalties = [GroupNormL2(l, somatic_grpidx) for l in lambdas]

score = ParProx.cross_validate(U, somatic_predictors, 
    survival_event, survival_time, penalties, 5; T=Float64, A=Array)

10	(-0.04472522319121207, Inf, 2.0)
20	(-0.044723958842517474, 1.2102227424703307e-6, 2.0)
  2.152296 seconds (4.96 M allocations: 269.527 MiB, 1.58% gc time)
10	(-0.04472269647652537, Inf, 2.0)
20	(-0.04472143608980039, 1.206433295455113e-6, 2.0)
  0.144078 seconds (1.25 k allocations: 11.932 MiB)
10	(-0.04472017767891388, Inf, 2.0)
20	(-0.044718921240443715, 1.202656948795527e-6, 2.0)
  0.180938 seconds (1.13 k allocations: 11.923 MiB, 24.44% gc time)
10	(-0.044717666770974385, Inf, 2.0)
20	(-0.044716414267097014, 1.1988936521584698e-6, 2.0)
  0.138394 seconds (1.25 k allocations: 11.932 MiB)
10	(-0.04471516372540919, Inf, 2.0)
20	(-0.04471391514251518, 1.1951433554350747e-6, 2.0)
  0.142427 seconds (1.13 k allocations: 11.923 MiB)
10	(-0.04471266851502568, Inf, 2.0)
20	(-0.04471142383955796, 1.1914060087008697e-6, 2.0)
  0.142674 seconds (1.25 k allocations: 11.932 MiB)
10	(-0.04470896951697031, Inf, 18.0)
20	(-0.044706614983195204, 2.2537751186168218e-6, 18.0)
  0.143495 seconds (1

20	(-0.044826976324422105, 2.650701541406773e-5, 1284.0)
  0.138166 seconds (1.13 k allocations: 11.923 MiB)
10	(-0.0447863236448905, Inf, 2341.0)
20	(-0.04474809964413228, 3.658681051560311e-5, 2315.0)
  0.139249 seconds (1.13 k allocations: 11.923 MiB)
10	(-0.044694611030451685, Inf, 4222.0)
20	(-0.04464397359747082, 4.847338831284696e-5, 4190.0)
  0.138614 seconds (1.25 k allocations: 11.932 MiB)
10	(-0.04457622340581936, Inf, 6835.0)
20	(-0.044511550986214087, 6.191642356105398e-5, 6814.0)
  0.140459 seconds (1.13 k allocations: 11.923 MiB)
10	(-0.04442967365354533, Inf, 9378.0)
20	(-0.04435095714254198, 7.53736188634587e-5, 9368.0)
  0.138825 seconds (1.13 k allocations: 11.923 MiB)
10	(-0.044256691718612524, Inf, 11661.0)
20	(-0.04416559183479002, 8.724658668595498e-5, 11646.0)
  0.138446 seconds (1.25 k allocations: 11.932 MiB)
10	(-0.04406149165162268, Inf, 13501.0)
20	(-0.043960529919589035, 9.671029616552857e-5, 13434.0)
  0.135659 seconds (1.13 k allocations: 11.923 MiB)
10	

21×5 Array{Float64,2}:
 0.631647  0.653762  0.681115  0.644217  0.584255
 0.631218  0.653762  0.680672  0.644217  0.586466
 0.631647  0.653762  0.681115  0.644217  0.584255
 0.631647  0.652882  0.681115  0.644217  0.584255
 0.631218  0.652882  0.680672  0.644682  0.585139
 0.631218  0.654201  0.681115  0.644217  0.584255
 0.631218  0.653322  0.68023   0.645146  0.585139
 0.629503  0.653762  0.681115  0.644217  0.585139
 0.629931  0.653322  0.681115  0.644217  0.585582
 0.628216  0.653762  0.681115  0.644217  0.585139
 0.626501  0.654641  0.682441  0.644682  0.586908
 0.622213  0.654201  0.681557  0.642824  0.590004
 0.616209  0.656841  0.679345  0.640966  0.592658
 0.607633  0.661241  0.679345  0.63725   0.5931
 0.596484  0.66696   0.678903  0.635857  0.597966
 0.587479  0.671359  0.671827  0.627497  0.598408
 0.58705   0.67004   0.6705    0.619136  0.59885
 0.580617  0.663     0.67448   0.60288   0.593543
 0.577187  0.653322  0.669173  0.601022  0.589562
 0.577187  0.652442  0.670057 

The number of unpenalized variables are automatically detected from the penalty structure. 

Simple lasso penalty could be used by replacing line 5 above with 
```julia
penalties = [NormL1(l; unpen=2) for l in lambdas]
```
For lasso penalty, the user should provide the number of unpenalized variables at the last columns as a keyword argument.

The resulting c-index for each fold is listed as:

In [20]:
score 

21×5 Array{Float64,2}:
 0.631647  0.653762  0.681115  0.644217  0.584255
 0.631218  0.653762  0.680672  0.644217  0.586466
 0.631647  0.653762  0.681115  0.644217  0.584255
 0.631647  0.652882  0.681115  0.644217  0.584255
 0.631218  0.652882  0.680672  0.644682  0.585139
 0.631218  0.654201  0.681115  0.644217  0.584255
 0.631218  0.653322  0.68023   0.645146  0.585139
 0.629503  0.653762  0.681115  0.644217  0.585139
 0.629931  0.653322  0.681115  0.644217  0.585582
 0.628216  0.653762  0.681115  0.644217  0.585139
 0.626501  0.654641  0.682441  0.644682  0.586908
 0.622213  0.654201  0.681557  0.642824  0.590004
 0.616209  0.656841  0.679345  0.640966  0.592658
 0.607633  0.661241  0.679345  0.63725   0.5931
 0.596484  0.66696   0.678903  0.635857  0.597966
 0.587479  0.671359  0.671827  0.627497  0.598408
 0.58705   0.67004   0.6705    0.619136  0.59885
 0.580617  0.663     0.67448   0.60288   0.593543
 0.577187  0.653322  0.669173  0.601022  0.589562
 0.577187  0.652442  0.670057 

We select the penalty parameter that maximizes the mean c-index as following:

In [21]:
lambda_idx = argmax(mean(score; dims=2)[:])
lambda = lambdas[lambda_idx]

7.943282347242822e-5

Now we fit the model with the selected value of lambda. 

In [22]:
p = GroupNormL2(lambda, somatic_grpidx)
U = ParProx.COXUpdate(; maxiter=20000, step=20, tol=1e-6, verbose=true)
V = ParProx.COXVariables{Float64, Array}(somatic_predictors, survival_event, survival_time, p; eval_obj=true)
@time ParProx.fit!(U, V)

20	(-0.0585762628073384, Inf, 2.0)
40	(-0.05857394142183881, 2.1929365618731874e-6, 2.0)
60	(-0.05857162508063111, 2.1881761732719335e-6, 2.0)
80	(-0.05856931377169719, 2.1834271066131557e-6, 2.0)
100	(-0.05856700748305059, 2.1786893321792244e-6, 2.0)
120	(-0.05856470620273639, 2.1739628203303116e-6, 2.0)
140	(-0.05856240991883119, 2.1692475414650616e-6, 2.0)
160	(-0.05856011861944299, 2.1645434660664837e-6, 2.0)
180	(-0.05855783229271107, 2.159850564774059e-6, 2.0)
200	(-0.05855555092680593, 2.155168808232982e-6, 2.0)
220	(-0.05855327450992917, 2.150498167238373e-6, 2.0)
240	(-0.058551003030313443, 2.1458386126172936e-6, 2.0)
260	(-0.05854873647622236, 2.141190115281191e-6, 2.0)
280	(-0.05854647483595033, 2.136552646284898e-6, 2.0)
300	(-0.058544218097822565, 2.1319261767086455e-6, 2.0)
320	(-0.05854196625019495, 2.1273106777170634e-6, 2.0)
340	(-0.05853971928145392, 2.1227061206116253e-6, 2.0)
360	(-0.05853747718001642, 2.11811247672577e-6, 2.0)
380	(-0.05853523993432983, 2.113529717

2980	(-0.058281767680847155, 1.6006984563844955e-6, 2.0)
3000	(-0.05828007726554337, 1.5973231851353244e-6, 2.0)
3020	(-0.05827839041671177, 1.5939556612686307e-6, 2.0)
3040	(-0.05827670712615712, 1.5905958652499209e-6, 2.0)
3060	(-0.05827502738570501, 1.5872437775115032e-6, 2.0)
3080	(-0.05827335118720174, 1.583899378537725e-6, 2.0)
3100	(-0.05827167852251414, 1.5805626490288973e-6, 2.0)
3120	(-0.05827000938352973, 1.5772335695144432e-6, 2.0)
3140	(-0.058268343762156526, 1.57391212070697e-6, 2.0)
3160	(-0.058266681650322956, 1.5705982833908037e-6, 2.0)
3180	(-0.05826502303997793, 1.567292038303969e-6, 2.0)
3200	(-0.05826336792309066, 1.5639933663348955e-6, 2.0)
3220	(-0.05826171629165068, 1.5607022483716135e-6, 2.0)
3240	(-0.05826006813766773, 1.5574186653869951e-6, 2.0)
3260	(-0.05825842345317174, 1.5541425983862993e-6, 2.0)
3280	(-0.058256782230212835, 1.5508740283678351e-6, 2.0)
3300	(-0.05825514446086109, 1.5476129365590108e-6, 2.0)
3320	(-0.058253510137206664, 1.544359304049148e-

5920	(-0.05806765650386324, 1.178451391627358e-6, 2.0)
5940	(-0.05806641218566703, 1.1760303340879074e-6, 2.0)
5960	(-0.05806517042487784, 1.1736146542740817e-6, 2.0)
5980	(-0.05806393121581054, 1.171204339112415e-6, 2.0)
6000	(-0.05806269455279385, 1.168799375555468e-6, 2.0)
6020	(-0.058061460430170346, 1.1663997505424807e-6, 2.0)
6040	(-0.05806022884229634, 1.164005451137092e-6, 2.0)
6060	(-0.058058999783541954, 1.161616464337158e-6, 2.0)
6080	(-0.05805777324829095, 1.1592327772780511e-6, 2.0)
6100	(-0.05805654923094081, 1.1568543770490372e-6, 2.0)
6120	(-0.0580553277259026, 1.1544812508375528e-6, 2.0)
6140	(-0.05805410872760106, 1.1521133857718127e-6, 2.0)
6160	(-0.05805289223047443, 1.1497507691372278e-6, 2.0)
6180	(-0.058051678228974515, 1.1473933881468723e-6, 2.0)
6200	(-0.05805046671756656, 1.1450412301382291e-6, 2.0)
6220	(-0.05804925769072933, 1.1426942823698869e-6, 2.0)
6240	(-0.05804805114295496, 1.1403525322576358e-6, 2.0)
6260	(-0.058046847068748984, 1.1380159671646068e-6,

The tuned value selects the most sparse model possible.

We obtain the indexes of variables with nonzero coefficients:

In [23]:
nonzero_idxs = (1:length(V.β))[V.β .!= 0];

The selected variables, along with the coefficients can be printed as below:

In [24]:
for (variable_name, value) in zip(somatic_variable_names[nonzero_idxs], V.β[nonzero_idxs])
    println(variable_name, "\t", value)
end

age_at_diagnosis								0.12933261107256303
gender								0.06219181385721187


### (optional) Analysis using GPU

For using GPU, as stated before, the package CUDA.jl is required. 

Cross validation on GPU is performed by the following code. 

In [25]:
using CUDA, Adapt

The following identifies the GPU we are running the code on.

In [26]:
CUDA.device()

CuDevice(0): TITAN V

It takes only a minor modification to run the cross validation on GPU. 

In [27]:
Random.seed!(222)
T = Float64
A = CuArray
U = ParProx.COXUpdate(; maxiter=10000, step=10, tol=5e-4, verbose=true)
lambdas = 10 .^ (range(-5, stop=-7, length=21)) # 21 values equally log-spaced in 10^-5 .. 10^-7
penalties = [GroupNormL2{T,A}(l, somatic_grpidx) for l in lambdas]

score = ParProx.cross_validate(U, somatic_predictors, 
    survival_event, survival_time, penalties, 5; T=T, A=A)

10	(-0.0446867720206526, Inf, 585.0)
20	(-0.044652587072046626, 3.27237485734705e-5, 587.0)
 13.583053 seconds (24.76 M allocations: 1.253 GiB, 2.51% gc time)
10	(-0.04460053748032986, Inf, 1143.0)
20	(-0.04455428048303685, 4.428395743265993e-5, 1136.0)
  0.029868 seconds (13.49 k allocations: 20.141 MiB)
10	(-0.04448706724113553, Inf, 1996.0)
20	(-0.044426277304298635, 5.820414342101704e-5, 1955.0)
  0.037921 seconds (13.74 k allocations: 20.154 MiB)
10	(-0.04433950339127302, Inf, 3622.0)
20	(-0.044259512460611265, 7.660062437283778e-5, 3559.0)
  0.031738 seconds (13.63 k allocations: 20.153 MiB)
10	(-0.04414972646035248, Inf, 6158.0)
20	(-0.044046930698031714, 9.845894786744629e-5, 6158.0)
  0.037705 seconds (13.62 k allocations: 20.147 MiB)
10	(-0.04391194913313793, Inf, 8986.0)
20	(-0.043784281952638754, 0.00012231184422546124, 8955.0)
  0.041364 seconds (13.75 k allocations: 20.155 MiB)
10	(-0.04362685454875632, Inf, 11671.0)
20	(-0.04347737421797781, 0.00014325210538516518, 11605

20	(-0.04258000003383254, 0.00011141063420297835, 18203.0)
  0.025371 seconds (13.11 k allocations: 20.143 MiB)
10	(-0.04246291059885666, Inf, 18458.0)
20	(-0.04234834139432571, 0.0001099145074454601, 18435.0)
  0.017332 seconds (12.99 k allocations: 20.130 MiB)
10	(-0.042233436368776196, Inf, 18687.0)
20	(-0.0421209195807111, 0.00010796903310449761, 18676.0)
  0.017345 seconds (12.99 k allocations: 20.130 MiB)
10	(-0.042008513860035, Inf, 18812.0)
20	(-0.04189836572582672, 0.00010571869371495495, 18800.0)
  0.035543 seconds (24.51 k allocations: 20.358 MiB, 23.08% gc time)
10	(-0.04178865316358038, Inf, 20812.0)
20	(-0.041681072851778724, 0.00010327567103349352, 20783.0)
  0.016193 seconds (12.99 k allocations: 20.131 MiB)
10	(-0.044814659304232916, Inf, 56.0)
20	(-0.04481070000299285, 3.7894914744403903e-6, 56.0)
  0.023115 seconds (13.12 k allocations: 20.145 MiB)
10	(-0.04480311142303066, Inf, 140.0)
20	(-0.044796060115894064, 6.748979447541888e-6, 140.0)
  0.018357 seconds (12.99 

21×5 Array{Float64,2}:
 0.599057  0.655961  0.682441  0.640037  0.586466
 0.57976   0.66256   0.683768  0.632606  0.59885
 0.553602  0.672679  0.678903  0.629354  0.605484
 0.54331   0.66784   0.670057  0.629819  0.607696
 0.53988   0.67004   0.659     0.617278  0.599735
 0.545026  0.6696    0.653251  0.603344  0.587793
 0.538165  0.656841  0.647059  0.590804  0.581601
 0.540738  0.646282  0.642194  0.588017  0.569217
 0.542453  0.639243  0.634675  0.572225  0.56789
 0.552744  0.626925  0.625387  0.567116  0.568775
 0.551887  0.619006  0.619195  0.556897  0.570102
 0.552316  0.612407  0.616984  0.557362  0.574525
 0.557461  0.608887  0.61433   0.553646  0.576736
 0.562178  0.602728  0.614772  0.552253  0.572313
 0.560034  0.598768  0.609907  0.552717  0.568333
 0.558319  0.592169  0.604157  0.550395  0.570102
 0.560463  0.58161   0.602831  0.548537  0.564352
 0.561321  0.575451  0.59885   0.544357  0.562583
 0.561321  0.572371  0.601504  0.540176  0.562141
 0.564751  0.571051  0.599292

It can be seen that analysis on GPU is 6-8 times faster except for the very first fit where the functions are compiled. It may be desirable to use `T=Float32` to reduce memory usage of the GPU. 

In [28]:
lambda_idx = argmax(mean(score; dims=2)[:])
lambda = lambdas[lambda_idx]

1.0e-5

In [29]:
p = GroupNormL2{T,A}(lambda, somatic_grpidx)
U = ParProx.COXUpdate(; maxiter=20000, step=20, tol=1e-6, verbose=true)
V = ParProx.COXVariables{T,A}(adapt(A{T}, somatic_predictors), 
    adapt(A{T}, survival_event), 
    adapt(A{T}, survival_time), p; eval_obj=true)
@time ParProx.fit!(U, V)

20	(-0.05856762640035516, Inf, 64.0)
40	(-0.058557811848852025, 9.271625407021259e-6, 64.0)
60	(-0.05854891761891099, 8.402285234999274e-6, 64.0)
80	(-0.05854077218925279, 7.694960715922886e-6, 64.0)
100	(-0.05853324085171504, 7.114880522498248e-6, 63.0)
120	(-0.058526208157047, 6.643855025835135e-6, 63.0)
140	(-0.058519607025598504, 6.236191946450412e-6, 63.0)
160	(-0.058513177348886204, 6.074252876476307e-6, 47.0)
180	(-0.058506719671932536, 6.100742521190199e-6, 43.0)
200	(-0.058500456084850404, 5.9174155723086635e-6, 43.0)
220	(-0.05849442097393898, 5.701599169381907e-6, 43.0)
240	(-0.05848859223611299, 5.506660977498699e-6, 43.0)
260	(-0.05848295073285316, 5.329800783209927e-6, 43.0)
280	(-0.058477479803838855, 5.168677764705353e-6, 43.0)
300	(-0.058472164873476784, 5.021322750331123e-6, 43.0)
320	(-0.058466993130185274, 4.8860694996408254e-6, 43.0)
340	(-0.058461953263233654, 4.761500341209305e-6, 43.0)
360	(-0.05845703242440952, 4.649068099498043e-6, 41.0)
380	(-0.05845208728264

2940	(-0.05808955688646256, 1.8449027836459418e-6, 38.0)
2960	(-0.05808761266927155, 1.837482234670791e-6, 38.0)
2980	(-0.058085676249031847, 1.8301166750218643e-6, 38.0)
3000	(-0.05808374756830403, 1.8228053613450795e-6, 38.0)
3020	(-0.058081826570419166, 1.8155475660016962e-6, 38.0)
3040	(-0.05807991319946284, 1.8083425764521424e-6, 38.0)
3060	(-0.0580780074002595, 1.8011896949087079e-6, 38.0)
3080	(-0.05807610911835742, 1.7940882378111642e-6, 38.0)
3100	(-0.05807421830001398, 1.7870375354925574e-6, 38.0)
3120	(-0.05807233489218156, 1.7800369316089027e-6, 38.0)
3140	(-0.05807045884249355, 1.7730857830213764e-6, 38.0)
3160	(-0.058068590099251076, 1.7661834591472986e-6, 38.0)
3180	(-0.0580667286114099, 1.7593293417504965e-6, 38.0)
3200	(-0.058064874328567787, 1.7525228245480353e-6, 38.0)
3220	(-0.05806302720095211, 1.7457633129481075e-6, 38.0)
3240	(-0.058061187179407996, 1.7390502235714901e-6, 38.0)
3260	(-0.05805935421538657, 1.7323829841140249e-6, 38.0)
3280	(-0.05805752826093364, 1

5860	(-0.05786806721017658, 1.122367687841813e-6, 38.0)
5880	(-0.057866883458948644, 1.1189982846095814e-6, 38.0)
5900	(-0.05786570325880323, 1.1156426962089163e-6, 38.0)
5920	(-0.057864526595213656, 1.1123008286938531e-6, 38.0)
5940	(-0.05786335345375179, 1.1089725889767209e-6, 38.0)
5960	(-0.05786218382008694, 1.1056578850642914e-6, 38.0)
5980	(-0.05786101767998491, 1.1023566258150965e-6, 38.0)
6000	(-0.05785985501930698, 1.0990687210706274e-6, 38.0)
6020	(-0.057858695824008946, 1.0957940815831936e-6, 38.0)
6040	(-0.05785754008014011, 1.0925326190421737e-6, 38.0)
6060	(-0.05785638777384239, 1.0892842459887547e-6, 38.0)
6080	(-0.0578552388913493, 1.0860488759208947e-6, 38.0)
6100	(-0.05785409341898511, 1.082826423149027e-6, 38.0)
6120	(-0.05785295134316388, 1.0796168028682259e-6, 38.0)
6140	(-0.05785181265038853, 1.076419931158217e-6, 38.0)
6160	(-0.057850677327249994, 1.0732357249177961e-6, 38.0)
6180	(-0.05784954531932676, 1.070102953904199e-6, 42.0)
6200	(-0.0578484163668708, 1.067

In [30]:
nonzero_idxs = (1:length(V.β))[V.β .!= 0];

┌ Warning: Performing scalar operations on GPU arrays: This is very slow, consider disallowing these operations with `allowscalar(false)`
└ @ GPUArrays /home/kose/.julia/packages/GPUArrays/jhRU7/src/host/indexing.jl:43


In [31]:
for (variable_name, value) in zip(somatic_variable_names[nonzero_idxs], V.β[nonzero_idxs])
    println(variable_name, "\t", value)
end

Q6NUM9	NA	NA	NA	LU	RETSAT	ENSG00000042445	LU	-0.0016328941806089774
Q5JXM2	115	342	228	Methyltransf_22	METTL24	ENSG00000053328	PIU	-0.0016328941806089774
P27815	NA	NA	NA	LU	PDE4A	ENSG00000065989	LU	-0.0036015015186720094
Q12888	NA	NA	NA	NCU	TP53BP1	ENSG00000067369	NCU	-0.003260300192141836
Q9NPF4	23	301	162	Peptidase_M22	OSGEP	ENSG00000092094	PIU	-0.0005769446374899101
P25490	NA	NA	NA	LU	YY1	ENSG00000100811	LU	-0.0036015015186720094
O00264	NA	NA	NA	LU	PGRMC1	ENSG00000101856	LU	-0.0016328941806089774
Q9HC57	NA	NA	NA	LU	WFDC1	ENSG00000103175	LU	-0.0036015015186720094
Q9NUM4	NA	NA	NA	NCU	TMEM106B	ENSG00000106460	NCU	-0.0016328941806089774
Q9Y2J4	NA	NA	NA	LU	AMOTL2	ENSG00000114019	LU	-0.003260300192141836
Q9UI95	NA	NA	NA	LU	MAD2L2	ENSG00000116670	LU	-0.0003051353783785732
Q8TEH3	NA	NA	NA	LU	DENND1A	ENSG00000119522	LU	-0.0016328941806089774
Q8IX21	NA	NA	NA	LU	SLF2	ENSG00000119906	LU	-0.0005769446374899101
Q9BUF7	NA	NA	NA	NCU	CRB3	ENSG00000130545	NCU	-0.0005769446374899101
P36222	NA	NA	NA	NC

## Example 2: Logistic regression with overlapping groups

In this example, we perform logistic regression with overlapping group lasso penalties. This package provides a specialized interface for overlapping groups. We use mRNA gene expression signature to classify breast cancer patients undergoing chemotherapy with anthracycline and neoadjuvant agents into two classes. Group of variables is defined by the pathway and GO terms. Genes not belonging to any of the pathways is included as a singleton group. 

In latent group lasso model, variables in multiple groups are replicated each time, each replication corresponding to the contribution of a variable as a member of each group. We estimate the corresponding coefficient for each replication, and the sum of these replicated coefficients is the total coefficient for that variable. This is implemented as a lazy multiplication between the original data and a sparse matrix, reducing the memory usage. 


For this analysis, we have four data files.

- `exprdata_set2_cut.txt.gz` contains the gene expression data of the subjects. 
- `sample_set2.txt.gz` contains the clinical information of the subjects.
- `PATHWAYgroup.txt.gz` contains the information on the group structure: membership of each gene in a pathway. 
- `gene_info.txt.gz` matches gene symbols to their functions.

For the preprocessing, we first read the gene expression and clinical data, then parse `PATHWAYgroup.txt.gz` to determine the group structure. In the process, we also attach the functions to each of the genes and pathways. Then, we construct variable names from the group structure and the functions.

### Reading covariates

The following lines read the gene expression data. The first column of the file `exprdata_set2_cut.txt.gz` has the gene names, and the other columns represent the normalized gene expression level of each patient. We extract the gene names for later use.

In [32]:
brca_data = DataFrame!(CSV.File(transcode(GzipDecompressor, Mmap.mmap(
            ParProx.datadir("exprdata_set2_cut.txt.gz")))))
gene_names = brca_data[!, :Gene]

1043-element Array{String,1}:
 "RFC2"
 "PAX8"
 "THRA"
 "EPHB3"
 "CFL1"
 "YY1"
 "ZPR1"
 "RHOA"
 "GUK1"
 "DSP"
 "RAD21"
 "SF3B2"
 "NDRG1"
 ⋮
 "ORAI3"
 "FAM174B"
 "ZNF335"
 "SEH1L"
 "LOC389906"
 "KIF18B"
 "RACGAP1"
 "CASP8AP2"
 "KLK5"
 "OR7E47P"
 "SCAF4"
 "SNHG17"

Now we load the covariates. Be carful when matching the subject information on the gene expression data and clinical data. We gather penalized variables and unpenalized variables separately. 

In [33]:
# patient names in the gene expression file
patient_barcode = map(x -> string(x), names(brca_data)[2:end])

X = collect(transpose(convert(Matrix, brca_data[:, 2:end])))

subject_data = CSV.File(transcode(GzipDecompressor, Mmap.mmap(
            ParProx.datadir("sample_set2.txt.gz")))) |> DataFrame
normalize(x) = (x .- mean(x; dims=1))./ std(x; dims=1)
Age  = normalize(subject_data[!, :Age])
interceptcol = ones(size(X, 1))

X_unpen = [Age interceptcol] 

y = subject_data[!, :PCR_outcome]

# match subject ids
subject_data = filter(x -> x[:GEO_accession] in patient_barcode, subject_data) 
@assert all(subject_data[!, :GEO_accession] .== patient_barcode)

### Building the groups

Now we read the group information. Each row of the file `PATHWAYgroup.txt.gz` lists a gene, a pathway id, function of the pathway and the gene symbol, indicating the membership of the gene in the pathway. We read the data file first:

In [34]:
group_info = DataFrame!(CSV.File(transcode(GzipDecompressor, Mmap.mmap(
            ParProx.datadir("PATHWAYgroup.txt.gz")))))
head(group_info)

,ENSG,Pathwayid,Function,Genesym
,String,String,String,String
1,ENSG00000148584,GO:0006397,mRNA processing,A1CF
2,ENSG00000148584,GO:0010467,gene expression,A1CF
3,ENSG00000148584,GO:0016554,cytidine to uridine editing,A1CF
4,ENSG00000148584,GO:0016556,mRNA modification,A1CF
5,ENSG00000148584,GO:0050821,protein stabilization,A1CF
6,ENSG00000175899,KEGG:04610,Complement and coagulation cascades,A2M


We create variables to store the group information. 

- `gene_names` (defined above): Each element lists the gene symbols in the analysis. Location in this list matches the index of each gene.
- `pathwayids`: A unique list of pathway ids. Location in this list determines the index of each pathway.
- `variable_to_groups`: Each element lists the group each variable is in.
- `group_to_variables`: Each element lists the variable each group includes.
- `genesym_to_variableidx`: Matches gene symbol to variable index.
- `pathwayid_to_groupidx`: Matches pathway id to group index. 

In [35]:
pathwayids = Vector{String}()
variable_to_groups = [Int[] for i in 1:length(gene_names)] 
group_to_variables = Vector{Vector{Int}}()
genesym_to_variableidx = Dict{String, Int}()
pathwayid_to_groupidx = Dict{String, Int}();

These dictionaries map pathwayids and gene symbols to their functions. 

In [36]:
pathwayid_to_pathwayfunction = Dict{String, String}()
genesym_to_genefunction = Dict{String, String}();

We build `genesym_to_variableidx` and `genesym_to_genefunction`: 

In [37]:
for (i, v) in enumerate(gene_names)
    genesym_to_variableidx[v] = i
end

gene_info = CSV.File(transcode(GzipDecompressor, Mmap.mmap(
            ParProx.datadir("gene_info.txt.gz")))) |> DataFrame
for r in eachrow(gene_info)
    genesym_to_genefunction[r[2]] = r[4]
end

`pathwayids`, `variable_to_groups`, `group_to_variables`, `pathwayid_to_groupidx`, and `pathwayid_to_pathwayfunction` are all built by parsing `group_info`.  

In [38]:
num_groups = 0
for r in eachrow(group_info)
    if !(r.Genesym in keys(genesym_to_variableidx)) # skip any gene not in our gene list
        continue
    end
    # get variable index from the gene symbol
    variable_idx = genesym_to_variableidx[r.Genesym]
    
    if !(r.Pathwayid in pathwayids) # if the pathway id is new ... #!haskey(pathwayid_to_genesym, r.Pathwayid) 
        # add a new group
        num_groups += 1
        push!(pathwayids, r.Pathwayid)
        push!(group_to_variables, Int[])
        pathwayid_to_groupidx[r.Pathwayid] = num_groups
        group_idx = num_groups
    else
        group_idx = pathwayid_to_groupidx[r.Pathwayid]
    end
    
    # insert membership information
    push!(variable_to_groups[variable_idx], group_idx)
    push!(group_to_variables[group_idx], variable_idx)
    
    # retrieve pathway information
    if !haskey(pathwayid_to_pathwayfunction, r.Pathwayid)
        pathwayid_to_pathwayfunction[r.Pathwayid] = r.Function
    else
        # if redundant, just check correctness
        @assert pathwayid_to_pathwayfunction[r.Pathwayid] == r.Function
    end
end



We also enter non-pathway genes as singleton groups.

In [39]:
cnt = 0
for (i, v) in enumerate(gene_names)
    if length(variable_to_groups[i]) > 0 # gene is in some of the groups
        continue
    else # gene is not in any of the groups
        num_groups += 1
        cnt += 1
        # update group info
        push!(variable_to_groups[i], num_groups)
        push!(group_to_variables, [i])
        
        # update variable names
        pathwayid_placeholder = 
            if haskey(genesym_to_genefunction, v)
                "Singleton of: $v"
            else # some genes do not appear in the gene function list
                "Singleton of: $v"
            end
        push!(pathwayids, pathwayid_placeholder)
    end
end
cnt

90

90 singleton groups were added!

### Constructing variable names

Here we create a vector of Strings for pathway ids and their functions.

First, we create a vector of variable names. We include age and intercept as nonpenalized variables. 

In [40]:
variable_names = [gene_names; "Age\t\t"; "intercept\t\t"]

1045-element Array{String,1}:
 "RFC2"
 "PAX8"
 "THRA"
 "EPHB3"
 "CFL1"
 "YY1"
 "ZPR1"
 "RHOA"
 "GUK1"
 "DSP"
 "RAD21"
 "SF3B2"
 "NDRG1"
 ⋮
 "ZNF335"
 "SEH1L"
 "LOC389906"
 "KIF18B"
 "RACGAP1"
 "CASP8AP2"
 "KLK5"
 "OR7E47P"
 "SCAF4"
 "SNHG17"
 "Age\t\t"
 "intercept\t\t"

We have 1045 variables for the analysis. 

Now we concatenate pathway names and their functions. 

In [41]:
pathways_with_functions = map(pathwayids) do x
    if haskey(pathwayid_to_pathwayfunction, x)
        "$x: $(pathwayid_to_pathwayfunction[x])"
    else
        x
    end
end

5719-element Array{String,1}:
 "KEGG:00280: Valine, leucine and isoleucine degradation"
 "KEGG:04727: GABAergic synapse"
 "KEGG:01100: Metabolic pathways"
 "KEGG:00250: Alanine, aspartate and glutamate metabolism"
 "KEGG:00640: Propanoate metabolism"
 "KEGG:00410: beta-Alanine metabolism"
 "KEGG:00650: Butanoate metabolism"
 "GO:0001666: response to hypoxia"
 "GO:0007268: synaptic transmission"
 "GO:0007269: neurotransmitter secretion"
 "GO:0007620: copulation"
 "GO:0007626: locomotory behavior"
 "GO:0009449: gamma-aminobutyric acid biosynthetic process"
 ⋮
 "Singleton of: TEX13B"
 "Singleton of: IQCG"
 "Singleton of: LOC100131532"
 "Singleton of: C6orf25"
 "Singleton of: TMEM14B"
 "Singleton of: FAM64A"
 "Singleton of: GPR124"
 "Singleton of: FAM174B"
 "Singleton of: LOC389906"
 "Singleton of: OR7E47P"
 "Singleton of: SCAF4"
 "Singleton of: SNHG17"

We now create replicated variable names: each replicated variable corresponds to a membership of a gene to a group. The name should include pathway information and group information. 

In [42]:
variable_names_replicated = String[]
cnt = 0
for i in 1:length(group_to_variables)
    for v in group_to_variables[i]
        pf = pathways_with_functions[i]
        g = gene_names[v]
        gt = genesym_to_genefunction[g]
        push!(variable_names_replicated, "$pf\t$g\t$gt")
    end
end

Finally, we append nonpenalized variables to the list. 

In [43]:
variable_names_replicated = [variable_names_replicated; "Age\t\t"; "intercept\t\t"] 

21965-element Array{String,1}:
 "KEGG:00280: Valine, leucine and isoleucine degradation\tABAT\t4-aminobutyrate aminotransferase"
 "KEGG:00280: Valine, leucine and isoleucine degradation\tAUH\tAU RNA binding protein/enoyl-CoA hydratase"
 "KEGG:00280: Valine, leucine and isoleucine degradation\tHMGCL\t3-hydroxymethyl-3-methylglutaryl-CoA lyase"
 "KEGG:00280: Valine, leucine and isoleucine degradation\tIVD\tisovaleryl-CoA dehydrogenase"
 "KEGG:00280: Valine, leucine and isoleucine degradation\tMCCC1\tmethylcrotonoyl-CoA carboxylase 1 (alpha)"
 "KEGG:00280: Valine, leucine and isoleucine degradation\tMCCC2\tmethylcrotonoyl-CoA carboxylase 2 (beta)"
 "KEGG:00280: Valine, leucine and isoleucine degradation\tMUT\tmethylmalonyl CoA mutase"
 "KEGG:04727: GABAergic synapse\tABAT\t4-aminobutyrate aminotransferase"
 "KEGG:04727: GABAergic synapse\tGABRA4\tgamma-aminobutyric acid (GABA) A receptor, alpha 4"
 "KEGG:04727: GABAergic synapse\tPRKCA\tprotein kinase C, alpha"
 "KEGG:04727: GABAergic syn

We have 21965 variables on the replicated space.

### Cross validation

Now we perform the 5-fold cross validation. The default criteria for cross validation is accuracy. It can be changed to area under the ROC curve (ROC) by adding a keyword parameter `criteria=auc`.

In [44]:
lambdas = 10 .^ (range(-6, stop=-9, length=31))

31-element Array{Float64,1}:
 1.0e-6
 7.943282347242822e-7
 6.30957344480193e-7
 5.011872336272725e-7
 3.981071705534969e-7
 3.162277660168379e-7
 2.5118864315095823e-7
 1.9952623149688787e-7
 1.584893192461114e-7
 1.2589254117941662e-7
 1.0e-7
 7.943282347242822e-8
 6.30957344480193e-8
 ⋮
 1.2589254117941661e-8
 1.0e-8
 7.943282347242822e-9
 6.309573444801943e-9
 5.011872336272715e-9
 3.981071705534969e-9
 3.1622776601683795e-9
 2.511886431509582e-9
 1.9952623149688828e-9
 1.584893192461111e-9
 1.2589254117941663e-9
 1.0e-9

For overlapping groups, `group_to_variables` is used as an input, along with `lambdas`, rather than the pre-constructed penalties. 

In [45]:
using Random
Random.seed!(222)
U = ParProx.LogisticUpdate(; maxiter=30000, step=20, tol=5e-4, verbose=true)
@time scores = ParProx.cross_validate(U, X, X_unpen, y, group_to_variables, lambdas, 5; T =Float64, criteria=auc)

20	(-0.6931299669675716, Inf, 5.0)
40	(-0.6931127550247755, 1.0165857380108762e-5, 5.0)
  0.758296 seconds (1.79 M allocations: 106.059 MiB)
20	(-0.6930909832098205, Inf, 35.0)
40	(-0.6930692158627235, 1.2856737865812443e-5, 35.0)
  0.178809 seconds (2.68 k allocations: 14.541 MiB, 77.86% gc time)
20	(-0.6930194596702821, Inf, 231.0)
40	(-0.6929697871677261, 2.9340454231704357e-5, 231.0)
  0.037132 seconds (2.44 k allocations: 14.522 MiB)
20	(-0.6928195220455069, Inf, 1272.0)
40	(-0.6926704275530877, 8.808241107792448e-5, 1272.0)
  0.038960 seconds (2.44 k allocations: 14.522 MiB)
20	(-0.6921737583844234, Inf, 6412.0)
40	(-0.6916903180042857, 0.0002857735691884926, 6366.0)
  0.038584 seconds (2.56 k allocations: 14.535 MiB)
20	(-0.6904786105516274, Inf, 14137.0)
40	(-0.6893248921581476, 0.000682946423648488, 14063.0)
60	(-0.6882256525617884, 0.0006511212495149772, 13960.0)
80	(-0.6871776821871006, 0.0006211381206330579, 13885.0)
100	(-0.6861777618018515, 0.0005930100656650706, 13770.0)

20	(-0.6860619233093129, Inf, 15500.0)
40	(-0.6844307802351707, 0.0009683645616559671, 15399.0)
60	(-0.6828845301419246, 0.0009188093808882105, 15192.0)
80	(-0.6814173417457926, 0.0008725902604338924, 15067.0)
100	(-0.6800241402201878, 0.0008292747063874045, 14969.0)
120	(-0.6787002374980747, 0.0007886474860372696, 14864.0)
140	(-0.677441478086922, 0.0007504043673632676, 14793.0)
160	(-0.6762437862934924, 0.0007145093113681216, 14749.0)
180	(-0.6751036999787957, 0.0006806064094486054, 14646.0)
200	(-0.6740175792526096, 0.0006488108247172616, 14622.0)
220	(-0.6729825204333719, 0.0006186907553401022, 14530.0)
240	(-0.6719946136822349, 0.0005908552234874468, 14428.0)
260	(-0.6710512484487431, 0.0005645339928189204, 14307.0)
280	(-0.6701494157020425, 0.000539971297311477, 14188.0)
300	(-0.6692870275562096, 0.0005166206479752917, 14068.0)
320	(-0.6684611420487239, 0.0004949983470826485, 13939.0)
  0.247090 seconds (19.87 k allocations: 116.209 MiB)
20	(-0.6669427502533459, Inf, 16722.0)
40	

140	(-0.6792750927625997, 0.000760437202334933, 14797.0)
160	(-0.6780605418323331, 0.0007237825453784619, 14739.0)
180	(-0.6769045690801553, 0.0006893491576637061, 14636.0)
200	(-0.6758032537618935, 0.0006571865257985842, 14553.0)
220	(-0.6747535051427538, 0.0006268078352523151, 14440.0)
240	(-0.6737520841274502, 0.0005983090475586132, 14334.0)
260	(-0.6727961208180352, 0.0005714762830437254, 14238.0)
280	(-0.6718830491383881, 0.0005461337024247022, 14160.0)
300	(-0.6710106252292528, 0.0005220935737710479, 14089.0)
320	(-0.6701763786795051, 0.0004994960774186689, 14023.0)
  0.257244 seconds (19.75 k allocations: 116.198 MiB, 2.19% gc time)
20	(-0.6686234551984976, Inf, 16884.0)
40	(-0.6671546652723159, 0.0008810159949627698, 16804.0)
60	(-0.6657641899411175, 0.0008347371972545259, 16688.0)
80	(-0.664446613244097, 0.0007916004553924713, 16627.0)
100	(-0.6631969745011138, 0.0007513474123280345, 16516.0)
120	(-0.6620108606273551, 0.0007136619271615023, 16392.0)
140	(-0.6608838013001387, 0

140	(-0.6747521843720313, 0.0006592806078063454, 16165.0)
160	(-0.6737028370640707, 0.0006269615398401872, 16057.0)
180	(-0.6727046439488229, 0.0005967539570472954, 15987.0)
200	(-0.671754256896371, 0.0005684968640165311, 15862.0)
220	(-0.6708489736780814, 0.0005418103207118838, 15830.0)
240	(-0.6699859847698667, 0.0005167641621457103, 15767.0)
260	(-0.6691625925518726, 0.0004932965917569906, 15668.0)
  0.203180 seconds (15.97 k allocations: 94.404 MiB)
20	(-0.667833518449433, Inf, 17825.0)
40	(-0.666575907003639, 0.0007546079602548839, 17755.0)
60	(-0.6653846856441733, 0.0007152830032209244, 17711.0)
80	(-0.6642552170497593, 0.000678663093762869, 17642.0)
100	(-0.6631832016960988, 0.000644556385951527, 17603.0)
120	(-0.6621648016072995, 0.0006126950154488242, 17539.0)
140	(-0.6611963935557498, 0.0005829581952540165, 17448.0)
160	(-0.6602745744370726, 0.000555220885069398, 17355.0)
180	(-0.6593960744405672, 0.0005294094701299834, 17234.0)
200	(-0.6585581588589724, 0.0005052072350428191

100	(-0.6599566271707529, 0.0006422284895526548, 17517.0)
120	(-0.6589408087940897, 0.000612329488356861, 17450.0)
140	(-0.657971996417883, 0.0005843357899288271, 17401.0)
160	(-0.657047208005836, 0.0005580941855964608, 17327.0)
180	(-0.6561636308818575, 0.0005335083487542204, 17280.0)
200	(-0.6553187845328041, 0.0005103828682109716, 17203.0)
220	(-0.6545100302588949, 0.0004888179939185372, 17119.0)
  0.172237 seconds (13.53 k allocations: 79.883 MiB)
20	(-0.6533415630937076, Inf, 18759.0)
40	(-0.6522298944416304, 0.0006728292810928247, 18674.0)
60	(-0.6511709930943246, 0.0006413032640074398, 18635.0)
80	(-0.6501612566062854, 0.0006119017059676748, 18587.0)
100	(-0.6491973719439718, 0.0005844568265213197, 18556.0)
120	(-0.6482763058611476, 0.0005588056320101916, 18547.0)
140	(-0.6473952524900547, 0.0005348160192650659, 18492.0)
160	(-0.6465515604495002, 0.0005123994054119581, 18441.0)
180	(-0.6457428964251424, 0.0004913671668365886, 18389.0)
  0.140659 seconds (11.21 k allocations: 65.

31×5 Array{Float64,2}:
 0.515088  0.722807  0.579659  0.523509  0.576491
 0.73193   0.719298  0.707681  0.674386  0.792281
 0.807018  0.728421  0.711238  0.729123  0.813333
 0.821053  0.738947  0.716927  0.764211  0.814035
 0.847018  0.738947  0.731863  0.797193  0.823158
 0.857544  0.738246  0.737553  0.812632  0.825965
 0.859649  0.740351  0.741821  0.815439  0.825965
 0.860351  0.743158  0.739687  0.81614   0.82386
 0.860351  0.740351  0.743243  0.81614   0.820351
 0.863158  0.740351  0.745377  0.818246  0.818246
 0.863158  0.741754  0.746088  0.820351  0.818947
 0.864561  0.742456  0.747511  0.821053  0.819649
 0.864561  0.741754  0.751067  0.820351  0.818947
 ⋮                                       
 0.866667  0.745965  0.756757  0.816842  0.815439
 0.865965  0.745263  0.756046  0.817544  0.81614
 0.865965  0.745263  0.756046  0.817544  0.815439
 0.866667  0.745263  0.756046  0.818246  0.815439
 0.867368  0.745965  0.756757  0.817544  0.81614
 0.868772  0.746667  0.757468  0.81614

We select the lambda with the highest mean AUC.

In [46]:
lambda_idx = argmax(mean(scores; dims=2)[:])
lambda = lambdas[lambda_idx]

1.2589254117941663e-9

Now fitting with the selected lambda:

In [47]:
U = ParProx.LogisticUpdate(; maxiter=30000, step=20, tol=1e-5, verbose=true)
V = ParProx.LogisticVariables{Float64}(X, X_unpen, y, lambda, group_to_variables)
@time ParProx.fit!(U, V)

20	(-0.6882550177051432, Inf, 21956.0)
40	(-0.6837195236951092, 0.0026937348805460793, 21956.0)
60	(-0.679511785655015, 0.0025053340357794455, 21965.0)
80	(-0.6756051154510824, 0.0023314981363499548, 21965.0)
100	(-0.6719749218217413, 0.002171201004250507, 21965.0)
120	(-0.6685985792479504, 0.0020234600555111815, 21965.0)
140	(-0.6654552969825821, 0.0018873411199106962, 21965.0)
160	(-0.6625259907792307, 0.0017619611480350075, 21965.0)
180	(-0.6597931556525485, 0.0016464913819986165, 21965.0)
200	(-0.657240747045071, 0.0015401555941878507, 21965.0)
220	(-0.6548540665157275, 0.0014422302108901696, 21965.0)
240	(-0.6526196543991636, 0.0013520425650367207, 21965.0)
260	(-0.6505251896861824, 0.0012689686446890043, 21965.0)
280	(-0.6485593943226994, 0.001192432235230872, 21965.0)
300	(-0.6467119433282783, 0.0011219029544943168, 21965.0)
320	(-0.6449733910173964, 0.0010568878015751134, 21965.0)
340	(-0.6433350875183512, 0.0009969381847248187, 21965.0)
360	(-0.6417891146766034, 0.000941639110

2820	(-0.5849315698077581, 0.00017249220677320135, 21965.0)
2840	(-0.5846590651948506, 0.0001719641902114774, 21965.0)
2860	(-0.5843874368253931, 0.00017144062313555384, 21965.0)
2880	(-0.5841166778718435, 0.00017092109270214965, 21965.0)
2900	(-0.5838467878147979, 0.00017040161909722803, 21965.0)
2920	(-0.5835777545108944, 0.00016988954482161282, 21965.0)
2940	(-0.5833095708293468, 0.00016938170935656532, 21965.0)
2960	(-0.5830422308916688, 0.0001688773252292393, 21965.0)
2980	(-0.5827757299526832, 0.000168375679473921, 21965.0)
3000	(-0.5825100597336647, 0.00016787900802557221, 21965.0)
3020	(-0.5822452141572677, 0.00016738592351376157, 21965.0)
3040	(-0.5819811872417167, 0.00016689636873078097, 21965.0)
3060	(-0.5817179733777046, 0.00016641011131079473, 21965.0)
3080	(-0.581455567140997, 0.00016592703719268997, 21965.0)
3100	(-0.5811939632913136, 0.0001654470329110641, 21965.0)
3120	(-0.5809331564401402, 0.00016497019504646679, 21965.0)
3140	(-0.5806731396585937, 0.00016449750111062

5580	(-0.5536307748259176, 0.00012322604235590287, 21965.0)
5600	(-0.5534397293528416, 0.00012298222419972004, 21965.0)
5620	(-0.5532490842495705, 0.00012273955620141573, 21965.0)
5640	(-0.5530588376753663, 0.0001224979824260568, 21965.0)
5660	(-0.5528689878064046, 0.0001222574927134961, 21965.0)
5680	(-0.5526795328312396, 0.00012201807981555583, 21965.0)
5700	(-0.5524904709506936, 0.00012177973654823152, 21965.0)
5720	(-0.5523018003777425, 0.00012154245579385474, 21965.0)
5740	(-0.5521135193374072, 0.00012130623049767626, 21965.0)
5760	(-0.5519256264921836, 0.00012107077943436935, 21965.0)
5780	(-0.5517381197109461, 0.0001208366146681992, 21965.0)
5800	(-0.5515509972068453, 0.00012060351508758974, 21965.0)
5820	(-0.5513642572520161, 0.00012037144336429783, 21965.0)
5840	(-0.5511778981301064, 0.00012014039275206514, 21965.0)
5860	(-0.5509919181361739, 0.00011991035656461847, 21965.0)
5880	(-0.5508063156955564, 0.00011968125144902367, 21965.0)
5900	(-0.5506210891464346, 0.00011945313424

8420	(-0.529815740413268, 9.697607206508535e-5, 21965.0)
8440	(-0.5296676126164053, 9.683659093057549e-5, 21965.0)
8460	(-0.5295197117931438, 9.669755945024401e-5, 21965.0)
8480	(-0.5293720370555168, 9.655906741391592e-5, 21965.0)
8500	(-0.5292245876162524, 9.642104924190586e-5, 21965.0)
8520	(-0.5290773626921534, 9.628350251664173e-5, 21965.0)
8540	(-0.5289303615040716, 9.614642483602213e-5, 21965.0)
8560	(-0.5287835832768776, 9.600981381510357e-5, 21965.0)
8580	(-0.5286370272394355, 9.58736670841572e-5, 21965.0)
8600	(-0.5284906926245738, 9.573798229050116e-5, 21965.0)
8620	(-0.5283445788831749, 9.560261698688192e-5, 21965.0)
8640	(-0.5281986851529057, 9.54677763347172e-5, 21965.0)
8660	(-0.5280530119548245, 9.533255518070836e-5, 21965.0)
8680	(-0.5279075577377208, 9.519830984999044e-5, 21965.0)
8700	(-0.5277623211562253, 9.506490602906052e-5, 21965.0)
8720	(-0.5276173014663006, 9.493195042079824e-5, 21965.0)
8740	(-0.5274724980366184, 9.479936946051546e-5, 21965.0)
8760	(-0.52732791

11340	(-0.5102513301002979, 8.067980682493315e-5, 21958.0)
11360	(-0.510129627593777, 8.059076803545763e-5, 21958.0)
11380	(-0.5100080689640932, 8.050197358691185e-5, 21958.0)
11400	(-0.5098866538107271, 8.041342246435062e-5, 21958.0)
11420	(-0.5097653820777172, 8.032488653503002e-5, 21958.0)
11440	(-0.5096442531381797, 8.023674404457931e-5, 21958.0)
11460	(-0.509523266482155, 8.014891768228993e-5, 21958.0)
11480	(-0.5094024217157916, 8.006133064630068e-5, 21958.0)
11500	(-0.5092817184468846, 7.997398194898721e-5, 21958.0)
11520	(-0.5091611562848667, 7.988687060746762e-5, 21958.0)
11540	(-0.5090407348407986, 7.979999564478394e-5, 21958.0)
11560	(-0.5089204537273603, 7.971335608924496e-5, 21958.0)
11580	(-0.5088003125588417, 7.962695097457776e-5, 21958.0)
11600	(-0.5086803109511341, 7.95407793397852e-5, 21958.0)
11620	(-0.508560448521721, 7.945484022900322e-5, 21958.0)
11640	(-0.5084407248896692, 7.936913269202053e-5, 21958.0)
11660	(-0.5083211397180086, 7.928362767819428e-5, 21958.0)
1

...

17020	(-0.48030010794373834, 6.243551292351736e-5, 21958.0)
17040	(-0.4802077590351828, 6.238915313869358e-5, 21958.0)
17060	(-0.48011548436943763, 6.234288251128971e-5, 21958.0)
17080	(-0.4800232838017952, 6.229670076918013e-5, 21958.0)
17100	(-0.4799311571879912, 6.225060764250857e-5, 21958.0)
17120	(-0.4798391044692291, 6.220454540230262e-5, 21958.0)
17140	(-0.47974712547402576, 6.215859022121226e-5, 21958.0)
17160	(-0.4796552200028119, 6.211276111584582e-5, 21958.0)
17180	(-0.47956338814302, 6.206686413560687e-5, 21958.0)
17200	(-0.47947162971232693, 6.20210816147461e-5, 21958.0)
17220	(-0.4793799443814039, 6.197551296489763e-5, 21958.0)
17240	(-0.47928833200903315, 6.193003107537835e-5, 21958.0)
17260	(-0.4791967924544267, 6.188463568431242e-5, 21958.0)
17280	(-0.47910532564468145, 6.183928092165356e-5, 21958.0)
17300	(-0.47901393137612863, 6.179405522419722e-5, 21958.0)
17320	(-0.47892260950575427, 6.174891761569619e-5, 21958.0)
17340	(-0.47883135989447173, 6.170386546918478e-5, 

19840	(-0.467952135962756, 5.666689776293529e-5, 21958.0)
19860	(-0.4678690092893649, 5.6630852524979686e-5, 21958.0)
19880	(-0.4677859401394579, 5.659486689123743e-5, 21958.0)
19900	(-0.46770292841656297, 5.65589407009563e-5, 21958.0)
19920	(-0.4676199740244663, 5.6523073796264625e-5, 21958.0)
19940	(-0.4675370768672142, 5.6487266017897776e-5, 21958.0)
19960	(-0.46745423684911086, 5.645151720792163e-5, 21958.0)
19980	(-0.46737145387471707, 5.641582720939267e-5, 21958.0)
20000	(-0.4672887278488509, 5.6380195864693726e-5, 21958.0)
20020	(-0.4672060586765851, 5.6344623017955444e-5, 21958.0)
20040	(-0.46712344626261704, 5.630910894274961e-5, 21950.0)
20060	(-0.4670408905085784, 5.6273655746588934e-5, 21950.0)
20080	(-0.4669583913256358, 5.6238256947432704e-5, 21950.0)
20100	(-0.4668759486198975, 5.6202916010643444e-5, 21950.0)
20120	(-0.46679356229772073, 5.616763278379649e-5, 21950.0)
20140	(-0.46671123226571404, 5.613240711296281e-5, 21950.0)
20160	(-0.4666289584307336, 5.60972388466072

22600	(-0.4569866408851296, 5.2193929151990075e-5, 21955.0)
22620	(-0.4569106414167825, 5.2164811064301384e-5, 21955.0)
22640	(-0.4568346882694441, 5.213573506312169e-5, 21955.0)
22660	(-0.4567587813751218, 5.210670104946565e-5, 21955.0)
22680	(-0.45668292066598576, 5.207770892335536e-5, 21955.0)
22700	(-0.45660710607436716, 5.2048758585916244e-5, 21955.0)
22720	(-0.4565313375327584, 5.2019849938234056e-5, 21955.0)
22740	(-0.4564556149738123, 5.199098288169799e-5, 21955.0)
22760	(-0.4563799383303419, 5.196215731807705e-5, 21955.0)
22780	(-0.45630430753531975, 5.193337314929157e-5, 21955.0)
22800	(-0.456228722521877, 5.190463027802319e-5, 21955.0)
22820	(-0.45615318322330434, 5.187592860626657e-5, 21955.0)
22840	(-0.45607768957304956, 5.184726803754044e-5, 21955.0)
22860	(-0.45600224158379676, 5.181859416007968e-5, 21955.0)
22880	(-0.4559268392292057, 5.178993377922774e-5, 21955.0)
22900	(-0.4558514823247327, 5.17613955735622e-5, 21955.0)
22920	(-0.45577617081840727, 5.1732888499685906e

25400	(-0.44676947827416985, 4.848466855756213e-5, 21957.0)
25420	(-0.44669937036678525, 4.846059162023634e-5, 21957.0)
25440	(-0.44662930064108625, 4.8436545331934607e-5, 21957.0)
25460	(-0.4465592692185595, 4.841241144899176e-5, 21957.0)
25480	(-0.4464892761356407, 4.8388248757545546e-5, 21957.0)
25500	(-0.446419321086423, 4.836429394841866e-5, 21957.0)
25520	(-0.44634940402140655, 4.834036977653589e-5, 21957.0)
25540	(-0.4462795248911584, 4.831647620358821e-5, 21957.0)
25560	(-0.4462096836463362, 4.829261317497327e-5, 21957.0)
25580	(-0.446139880237708, 4.826878062217078e-5, 21957.0)
25600	(-0.4460701146161654, 4.8244978467805946e-5, 21957.0)
25620	(-0.4460003867327308, 4.8221206629255696e-5, 21957.0)
25640	(-0.4459306965385616, 4.81974650209892e-5, 21957.0)
25660	(-0.4458610439849494, 4.817375355809939e-5, 21957.0)
25680	(-0.4457914290233174, 4.8150072157378344e-5, 21957.0)
25700	(-0.445721851605219, 4.8126420736588194e-5, 21957.0)
25720	(-0.4456523116823322, 4.810279921722634e-5, 

28160	(-0.4374360942233539, 4.5424155983040846e-5, 21963.0)
28180	(-0.4373708321644718, 4.5403772931583055e-5, 21963.0)
28200	(-0.43730560233055016, 4.53834131140139e-5, 21963.0)
28220	(-0.43724040499515815, 4.536286007922831e-5, 21963.0)
28240	(-0.4371752398881963, 4.5342492100637224e-5, 21963.0)
28260	(-0.43711010689466867, 4.532220128098414e-5, 21963.0)
28280	(-0.43704500597734564, 4.5301933517913604e-5, 21963.0)
28300	(-0.43697993709906846, 4.528168876771776e-5, 21963.0)
28320	(-0.43691490022275115, 4.5261466985437176e-5, 21963.0)
28340	(-0.43684989531137763, 4.5241268128028585e-5, 21963.0)
28360	(-0.4367849223280041, 4.5221092150849444e-5, 21963.0)
28380	(-0.43671998123575706, 4.520093901051685e-5, 21963.0)
28400	(-0.43665507199783365, 4.51808086635168e-5, 21963.0)
28420	(-0.4365901945775027, 4.5160701065508605e-5, 21963.0)
28440	(-0.4365253489381015, 4.514061617437761e-5, 21963.0)
28460	(-0.4364605350430398, 4.5120553945327754e-5, 21963.0)
28480	(-0.4363957528557952, 4.5100514336

In order to obtain the results in the original dimension, we need to obtain the "group matrix". Multiplying the group matrix by the _penalized_ part of the coefficients gives the aggregate coefficient for each penalized variable. Non-penalized variables (in this case, the last two variables) should be treated separately. 

In [48]:
_, grpmat, _ = ParProx.mapper_mat_idx(group_to_variables, length(gene_names));

In [49]:
size(grpmat)

(1043, 21963)

In [50]:
β_orig = vcat(grpmat * collect(V.β[1:end-2]), collect(V.β)[end-1:end]);

The following prints the coefficients in the replicated dimensions:

In [51]:
for (v, β) in zip(variable_names_replicated[V.β .!= 0], V.β[V.β .!= 0])
    println("$v\t$β")
end

KEGG:00280: Valine, leucine and isoleucine degradation	ABAT	4-aminobutyrate aminotransferase	0.0008323544134017853
KEGG:00280: Valine, leucine and isoleucine degradation	AUH	AU RNA binding protein/enoyl-CoA hydratase	-0.001459342688879147
KEGG:00280: Valine, leucine and isoleucine degradation	HMGCL	3-hydroxymethyl-3-methylglutaryl-CoA lyase	-0.0010615725170478396
KEGG:00280: Valine, leucine and isoleucine degradation	IVD	isovaleryl-CoA dehydrogenase	-0.0009348559171503025
KEGG:00280: Valine, leucine and isoleucine degradation	MCCC1	methylcrotonoyl-CoA carboxylase 1 (alpha)	8.306605556755142e-5
KEGG:00280: Valine, leucine and isoleucine degradation	MCCC2	methylcrotonoyl-CoA carboxylase 2 (beta)	-0.0007834944198441149
KEGG:00280: Valine, leucine and isoleucine degradation	MUT	methylmalonyl CoA mutase	0.002754888016414541
KEGG:04727: GABAergic synapse	ABAT	4-aminobutyrate aminotransferase	0.0008232735329612728
KEGG:04727: GABAergic synapse	GABRA4	gamma-aminobutyric acid (GABA) A receptor,

KEGG:00650: Butanoate metabolism	ABAT	4-aminobutyrate aminotransferase	0.0008342785686320111
KEGG:00650: Butanoate metabolism	ALDH5A1	aldehyde dehydrogenase 5 family, member A1	0.002331490731821373
KEGG:00650: Butanoate metabolism	BDH1	3-hydroxybutyrate dehydrogenase, type 1	-0.0002073436385173155
KEGG:00650: Butanoate metabolism	HMGCL	3-hydroxymethyl-3-methylglutaryl-CoA lyase	-0.0010643922027909467
GO:0001666: response to hypoxia	ABAT	4-aminobutyrate aminotransferase	0.0008334434534379154
GO:0001666: response to hypoxia	ADM	adrenomedullin	0.002238812559897346
GO:0001666: response to hypoxia	AGER	advanced glycosylation end product-specific receptor	-0.00017426454237396907
GO:0001666: response to hypoxia	BNIP3	BCL2/adenovirus E1B 19kDa interacting protein 3	0.002914150843654699
GO:0001666: response to hypoxia	CAV1	caveolin 1, caveolae protein, 22kDa	-0.0005354719528487098
GO:0001666: response to hypoxia	CCL2	chemokine (C-C motif) ligand 2	0.0008172213365582931
GO:0001666: response to h

GO:0035094: response to nicotine	BCL2	B-cell CLL/lymphoma 2	-5.08663682741707e-6
GO:0035094: response to nicotine	CHRNB2	cholinergic receptor, nicotinic, beta 2 (neuronal)	-0.000758327813289944
GO:0035094: response to nicotine	DRD2	dopamine receptor D2	-0.00023001114993561648
GO:0035094: response to nicotine	GPX1	glutathione peroxidase 1	0.0003867305456564964
GO:0035094: response to nicotine	HDAC2	histone deacetylase 2	-0.00118854149922459
GO:0035094: response to nicotine	HMOX1	heme oxygenase (decycling) 1	-0.0002107114039207381
GO:0035094: response to nicotine	PDX1	pancreatic and duodenal homeobox 1	0.0002469304872419897
GO:0035094: response to nicotine	STAR	steroidogenic acute regulatory protein	0.0008692233019067945
GO:0042135: neurotransmitter catabolic process	ABAT	4-aminobutyrate aminotransferase	0.0008384285830425878
GO:0042135: neurotransmitter catabolic process	ALDH5A1	aldehyde dehydrogenase 5 family, member A1	0.0023452776173856025
GO:0042493: response to drug	ABAT	4-aminobut

GO:0006355: regulation of transcription, DNA-templated	CEBPB	CCAAT/enhancer binding protein (C/EBP), beta	-0.0002745449759790138
GO:0006355: regulation of transcription, DNA-templated	CELSR2	cadherin, EGF LAG seven-pass G-type receptor 2	-0.0011360981432418863
GO:0006355: regulation of transcription, DNA-templated	CHAF1B	chromatin assembly factor 1, subunit B (p60)	0.006292511066352083
GO:0006355: regulation of transcription, DNA-templated	CHD3	chromodomain helicase DNA binding protein 3	0.00011917125375918101
GO:0006355: regulation of transcription, DNA-templated	CLOCK	clock circadian regulator	0.00020035738705752516
GO:0006355: regulation of transcription, DNA-templated	DAXX	death-domain associated protein	0.0008926960114535764
GO:0006355: regulation of transcription, DNA-templated	DLX6	distal-less homeobox 6	-0.00018007477071055246
GO:0006355: regulation of transcription, DNA-templated	E2F1	E2F transcription factor 1	0.0021001730920734304
GO:0006355: regulation of transcription, DNA

GO:0032367: intracellular cholesterol transport	ABCG1	ATP-binding cassette, sub-family G (WHITE), member 1	-0.0023414183670599823
GO:0032367: intracellular cholesterol transport	STAR	steroidogenic acute regulatory protein	0.0008955133587820492
GO:0032367: intracellular cholesterol transport	VPS4A	vacuolar protein sorting 4 homolog A (S. cerevisiae)	-0.0006857834708294202
GO:0033344: cholesterol efflux	ABCG1	ATP-binding cassette, sub-family G (WHITE), member 1	-0.0023351541238514494
GO:0033344: cholesterol efflux	CAV1	caveolin 1, caveolae protein, 22kDa	-0.0005351373609785363
GO:0033344: cholesterol efflux	SOAT1	sterol O-acyltransferase 1	-0.00040815997631377267
GO:0033700: phospholipid efflux	ABCG1	ATP-binding cassette, sub-family G (WHITE), member 1	-0.0023606226924747847
GO:0033993: response to lipid	ABCG1	ATP-binding cassette, sub-family G (WHITE), member 1	-0.002348053988443846
GO:0033993: response to lipid	GATA2	GATA binding protein 2	-0.00045468612400768345
GO:0033993: response t

...

KEGG:04022: cGMP-PKG signaling pathway	CREB1	cAMP responsive element binding protein 1	0.0006489619781641577
KEGG:04022: cGMP-PKG signaling pathway	GATA4	GATA binding protein 4	-0.0010834492017024245
KEGG:04022: cGMP-PKG signaling pathway	INSR	insulin receptor	-0.001348771581336535
KEGG:04022: cGMP-PKG signaling pathway	ITPR1	inositol 1,4,5-trisphosphate receptor, type 1	-0.0007704561243640072
KEGG:04022: cGMP-PKG signaling pathway	KCNMA1	potassium large conductance calcium-activated channel, subfamily M, alpha member 1	-0.0006479979619600895
KEGG:04022: cGMP-PKG signaling pathway	MEF2C	myocyte enhancer factor 2C	0.0009623584081566341
KEGG:04022: cGMP-PKG signaling pathway	PIK3CD	phosphatidylinositol-4,5-bisphosphate 3-kinase, catalytic subunit delta	-0.0007731994922275429
KEGG:04022: cGMP-PKG signaling pathway	PLCB1	phospholipase C, beta 1 (phosphoinositide-specific)	0.00015005615254737393
KEGG:04022: cGMP-PKG signaling pathway	RAF1	Raf-1 proto-oncogene, serine/threonine kinase	0.0014

GO:0006915: apoptotic process	DAXX	death-domain associated protein	0.000891037885543598
GO:0006915: apoptotic process	DHCR24	24-dehydrocholesterol reductase	-0.0017067530777823046
GO:0006915: apoptotic process	DIABLO	diablo, IAP-binding mitochondrial protein	-0.0005955950874973306
GO:0006915: apoptotic process	DOCK1	dedicator of cytokinesis 1	-0.0006838636074784749
GO:0006915: apoptotic process	DSG2	desmoglein 2	1.6684566336291924e-5
GO:0006915: apoptotic process	DSP	desmoplakin	-5.6663577730001117e-5
GO:0006915: apoptotic process	E2F1	E2F transcription factor 1	0.0020967558273057426
GO:0006915: apoptotic process	EDAR	ectodysplasin A receptor	0.0005937664990297259
GO:0006915: apoptotic process	ESPL1	extra spindle pole bodies homolog 1 (S. cerevisiae)	0.00019668379559849639
GO:0006915: apoptotic process	FAS	Fas cell surface death receptor	-3.3268354249467116e-5
GO:0006915: apoptotic process	FEM1B	fem-1 homolog b (C. elegans)	-0.0041294375883662495
GO:0006915: apoptotic process	FGF8	fibr

GO:0006805: xenobiotic metabolic process	UGDH	UDP-glucose 6-dehydrogenase	0.0005021831791150158
GO:0006805: xenobiotic metabolic process	UGP2	UDP-glucose pyrophosphorylase 2	0.0007852680716861067
GO:0007586: digestion	AKR1C1	aldo-keto reductase family 1, member C1	-0.0014971109160566132
GO:0007586: digestion	NMU	neuromedin U	0.0018630596659245915
GO:0007586: digestion	PGC	progastricsin (pepsinogen C)	0.0013304868788324748
GO:0007586: digestion	SLC15A1	solute carrier family 15 (oligopeptide transporter), member 1	-0.002777325068263748
GO:0007603: phototransduction, visible light	AKR1C1	aldo-keto reductase family 1, member C1	-0.0014807883341618933
GO:0007603: phototransduction, visible light	LRP8	low density lipoprotein receptor-related protein 8, apolipoprotein e receptor	-0.0009278360939371801
GO:0007603: phototransduction, visible light	METAP1	methionyl aminopeptidase 1	-0.0010119364799306915
GO:0007603: phototransduction, visible light	PRKCA	protein kinase C, alpha	-6.42321874417016

Excessive output truncated after 524293 bytes.

KEGG:05166: HTLV-I infection	E2F1	E2F transcription factor 1	0.002096507746263104
KEGG:05166: HTLV-I infection	E2F3	E2F transcription factor 3	0.0021608927453620713
KEGG:05166: HTLV-I infection	EGR1	early growth response 1	0.001092420152729813
KEGG:05166: HTLV-I infection	EGR2	early growth response 2	0.001437952547003632
KEGG:05166: HTLV-I infection	ETS2	v-ets avian erythroblastosis virus E26 oncogene homolog 2	-0.0010148671813836451
KEGG:05166: HTLV-I infection	FZD3	frizzled class receptor 3	0.0003783187873494029
KEGG:05166: HTLV-I infection	FZD7	frizzled class receptor 7	0.0010700104194553235
KEGG:05166: HTLV-I infection	GSK3B	glycogen synthase kinase 3 beta	0.0015481661412175235
KEGG:05166: HTLV-I infection	ICAM1	intercellular adhesion molecule 1	0.0007288551674977712
KEGG:05166: HTLV-I infection	MAD2L1	MAD2 mitotic arrest deficient-like 1 (yeast)	0.00031861014213334544
KEGG:05166: HTLV-I infection	MSX1	msh homeobox 1	-0.002767385544638793
KEGG:05166: HTLV-I infection	MYC	v-myc avia

The following prints the coefficients in the original dimensions:

In [52]:
for (v, β) in zip(variable_names[β_orig .!= 0], β_orig[β_orig .!= 0])
    println("$v\t$β")
end

RFC2	0.017828370354349202
PAX8	0.03684757874519625
THRA	0.028067107007347436
EPHB3	0.019420774902517743
CFL1	-0.030460550648403972
YY1	0.013402782876598915
ZPR1	-0.0041704594130534
RHOA	0.043220992206706325
GUK1	0.05284161532839049
DSP	-0.0008924333259685115
RAD21	-0.0016314685844209583
SF3B2	0.004117778492081896
NDRG1	-0.006543628992920163
CD63	-0.015885903765799944
DNAJB1	-0.0020740235707915316
XBP1	0.021161233616516405
SPTBN1	0.03684346496032824
HMGB1	-0.030756596050312508
GPX1	0.02123400737801506
NUMA1	0.021341452238237465
ARL6IP5	0.0010924161799196028
STMN1	0.007247922509085323
ODC1	0.00984689697218672
DAZAP2	-0.0007855012627521636
TMBIM6	-0.0022818769304895872
GSTP1	0.00815519724724144
ZNF207	-0.0027205566179267678
DHCR24	-0.03582951324451187
DEK	0.01752513149401217
HIF1A	-0.04980124402515669
SNX17	0.011478189129344094
CD9	-0.03889280343675502
JUP	-0.038198768007394045
TGM2	-0.0017271186611303288
MMP2	-0.02598193263059217
THBS1	-0.08446415723750327
TUFM	-0.006526015058534709
POLD

RAD51	-0.05959231669664289
CCNE2	0.018853635423271606
ORM1	-0.0027846959939131543
CENPE	0.03231574820149772
PSPH	-0.021255067676917736
KIT	0.02330305444052017
AUH	-0.008742765931883348
PRIM1	0.007640062297202148
SLC22A5	-0.02409020140972212
SERPINF2	0.014325953337234937
CRIP1	0.01716096384814977
ORC1	-0.005887435280780701
MOK	0.005371912559628551
PEX11A	0.006381301901495135
GLI3	-0.14256324992470182
KAL1	-0.0011795132247510995
GALK2	0.00012695444364892882
ESR1	-0.04057493084726799
AREG	0.05169677369466025
GPSM2	0.0071155939118706865
CXCL13	0.0783715582690652
EGR2	0.03901184423000848
PER2	0.03325074136646314
PGC	0.005304266152110364
LRP8	-0.020463024120600196
TFAP2C	0.02018951968784407
CDC14A	0.002323084411661582
IGFBP1	0.003091742143176985
SLC15A2	-0.0010016510545315745
S100A1	0.007943911280445212
DCT	-0.030649335406504684
BARD1	0.020946437515230742
AGTR1	0.012107886892131901
INPP4B	0.0043235597350753455
PDZK1	-0.0015817353024116022
CHEK1	0.01974575498226775
MLLT10	-0.00032716360192899

METAP1	-0.008021374838937058
SLC25A44	0.00034531985411083167
JMJD6	0.07225247109460833
C16orf45	0.0015129853607803163
RCHY1	-0.012047180085251727
BBS4	-0.024491286185357838
TCF7L2	0.022737387684181265
FAM171A1	-0.0008474461993496482
OBSL1	0.0004996753392981064
NCAPD3	-0.006291945324096308
CBS	0.027408147568601843
CKAP5	-0.00784421841174049
POLD3	0.04009422205891675
AXIN1	-0.040087790721424446
DNAJC16	-0.00010774179655897064
SMC5	-0.00840435524672013
NCAPH	0.0005077986713116518
KANK1	0.035704184864029716
NEDD4	0.08611271769809645
LOC100272216	-0.0004901303998456607
DNAJC2	-0.00785480274112632
HOXA10	-0.03585348281798796
ROBO1	-0.03381507510547673
TNNT1	0.002361670276473978
FOXC1	-0.0010032433396977912
YAP1	-0.005210933274387347
ATAD2B	-0.001240321054198613
SPDEF	0.031499791704416746
SOSTDC1	-0.012182241980776623
CLCN2	0.008450306755612699
CCNE1	0.013357563921641039
DNA2	-0.0005924515358255864
DLX5	-0.06904625505431022
ELOVL2	0.006080546124612895
HIPK2	0.028394484848613795
HOXA5	0.026519

### (optional) analysis using GPU

Once again, the analysis can be carried out using a GPU. At this time, data should be converted to `CuArray`s beforehand for overlapping groups. 

In [53]:
Random.seed!(222)
T = Float64
A = CuArray
U = ParProx.LogisticUpdate(; maxiter=30000, step=20, tol=5e-4, verbose=true)
lambdas = 10 .^ (range(-6, stop=-9, length=31))

scores = ParProx.cross_validate(U, 
    adapt(A{T}, X), adapt(A{T}, X_unpen), adapt(A{Int32}, y), group_to_variables, lambdas, 5; 
    T=Float64, criteria=auc)

20	(-0.6931299669675715, Inf, 5.0)
40	(-0.6931127550247755, 1.016585738004319e-5, 5.0)
  3.836978 seconds (5.09 M allocations: 304.254 MiB, 1.70% gc time)
20	(-0.6930909832098204, Inf, 35.0)
40	(-0.6930692158627236, 1.2856737865681294e-5, 35.0)
  0.019284 seconds (28.34 k allocations: 48.851 MiB)
20	(-0.693019459670282, Inf, 231.0)
40	(-0.6929697871677261, 2.934045423163878e-5, 231.0)
  0.040717 seconds (29.73 k allocations: 48.881 MiB)
20	(-0.6928195220455068, Inf, 1272.0)
40	(-0.6926704275530874, 8.808241107805568e-5, 1272.0)
  0.085327 seconds (38.40 k allocations: 49.061 MiB, 47.05% gc time)
20	(-0.6921737583844234, Inf, 6412.0)
40	(-0.6916903180042857, 0.0002857735691884926, 6366.0)
  0.018284 seconds (28.20 k allocations: 48.844 MiB)
20	(-0.6904786105516274, Inf, 14137.0)
40	(-0.6893248921581476, 0.000682946423648488, 14063.0)
60	(-0.6882256525617884, 0.0006511212495149772, 13960.0)
80	(-0.6871776821871006, 0.0006211381206330579, 13885.0)
100	(-0.6861777618018517, 0.0005930100656

100	(-0.6800241402201878, 0.0008292747063874045, 14969.0)
120	(-0.6787002374980746, 0.0007886474860373358, 14864.0)
140	(-0.6774414780869222, 0.0007504043673630689, 14793.0)
160	(-0.6762437862934926, 0.0007145093113681216, 14749.0)
180	(-0.6751036999787958, 0.0006806064094486717, 14646.0)
200	(-0.6740175792526096, 0.000648810824717328, 14622.0)
220	(-0.672982520433372, 0.0006186907553400357, 14530.0)
240	(-0.6719946136822349, 0.0005908552234875133, 14428.0)
260	(-0.6710512484487431, 0.0005645339928189204, 14307.0)
280	(-0.6701494157020427, 0.0005399712973113439, 14188.0)
300	(-0.6692870275562094, 0.0005166206479755577, 14068.0)
320	(-0.6684611420487239, 0.0004949983470825154, 13939.0)
  0.159736 seconds (263.94 k allocations: 391.534 MiB, 9.14% gc time)
20	(-0.6669427502533459, Inf, 16722.0)
40	(-0.665505513385753, 0.0008629433262404408, 16610.0)
60	(-0.6641438612476948, 0.0008182298236147547, 16527.0)
80	(-0.6628526067344715, 0.0007765297465293606, 16375.0)
100	(-0.6616267403085259, 0

240	(-0.6737520841274502, 0.0005983090475586132, 14334.0)
260	(-0.6727961208180354, 0.0005714762830435925, 14238.0)
280	(-0.6718830491383884, 0.0005461337024247022, 14160.0)
300	(-0.6710106252292527, 0.0005220935737712472, 14089.0)
320	(-0.6701763786795051, 0.0004994960774186024, 14023.0)
  0.170997 seconds (263.98 k allocations: 391.791 MiB, 9.37% gc time)
20	(-0.6686234551984978, Inf, 16884.0)
40	(-0.6671546652723159, 0.0008810159949628364, 16804.0)
60	(-0.6657641899411175, 0.0008347371972545259, 16688.0)
80	(-0.6644466132440972, 0.0007916004553924046, 16627.0)
100	(-0.6631969745011139, 0.0007513474123280344, 16516.0)
120	(-0.662010860627355, 0.0007136619271616359, 16392.0)
140	(-0.6608838013001387, 0.0006785901134890289, 16273.0)
160	(-0.6598120143427029, 0.0006457279186885819, 16189.0)
180	(-0.6587921885865662, 0.0006148001920636379, 16116.0)
200	(-0.6578210463559903, 0.0005857943670763306, 16060.0)
220	(-0.6568955128681048, 0.000558594963108675, 15977.0)
240	(-0.6560128756405132, 

160	(-0.6737028370640706, 0.0006269615398401873, 16057.0)
180	(-0.6727046439488228, 0.0005967539570472955, 15987.0)
200	(-0.671754256896371, 0.0005684968640164646, 15862.0)
220	(-0.6708489736780814, 0.0005418103207118838, 15830.0)
240	(-0.6699859847698665, 0.0005167641621458433, 15767.0)
260	(-0.6691625925518724, 0.0004932965917569906, 15668.0)
  0.121578 seconds (202.52 k allocations: 317.866 MiB, 5.98% gc time)
20	(-0.6678335184494327, Inf, 17825.0)
40	(-0.666575907003639, 0.0007546079602547506, 17755.0)
60	(-0.6653846856441733, 0.0007152830032209244, 17711.0)
80	(-0.6642552170497591, 0.0006786630937630026, 17642.0)
100	(-0.6631832016960986, 0.000644556385951527, 17603.0)
120	(-0.6621648016072994, 0.0006126950154487574, 17539.0)
140	(-0.6611963935557494, 0.0005829581952541502, 17448.0)
160	(-0.6602745744370727, 0.0005552208850691304, 17355.0)
180	(-0.659396074440567, 0.0005294094701301842, 17234.0)
200	(-0.6585581588589721, 0.0005052072350428862, 17172.0)
220	(-0.6577584863742947, 0.

220	(-0.654510030258895, 0.0004888179939183358, 17119.0)
  0.105957 seconds (174.52 k allocations: 269.021 MiB, 6.95% gc time)
20	(-0.6533415630937076, Inf, 18759.0)
40	(-0.6522298944416304, 0.0006728292810928247, 18674.0)
60	(-0.6511709930943247, 0.0006413032640073726, 18635.0)
80	(-0.6501612566062855, 0.0006119017059676747, 18587.0)
100	(-0.6491973719439719, 0.0005844568265213197, 18556.0)
120	(-0.6482763058611475, 0.0005588056320103262, 18547.0)
140	(-0.6473952524900548, 0.000534816019264931, 18492.0)
160	(-0.6465515604495004, 0.0005123994054119581, 18441.0)
180	(-0.6457428964251424, 0.0004913671668366561, 18389.0)
  0.077148 seconds (126.86 k allocations: 219.794 MiB)
20	(-0.6446783255038053, Inf, 19504.0)
40	(-0.6436620706076615, 0.0006182870033425826, 19475.0)
60	(-0.6426907122108307, 0.0005913215370429659, 19444.0)
80	(-0.6417611408924171, 0.0005662037523365652, 19406.0)
100	(-0.6408705194623594, 0.0005427737408247844, 19358.0)
120	(-0.640016233158196, 0.0005209011270079265, 193

31×5 Array{Float64,2}:
 0.515088  0.722807  0.579659  0.523509  0.576491
 0.73193   0.719298  0.707681  0.674386  0.792281
 0.807018  0.728421  0.711238  0.729123  0.813333
 0.821053  0.738947  0.716927  0.764211  0.814035
 0.847018  0.738947  0.731863  0.797193  0.823158
 0.857544  0.738246  0.737553  0.812632  0.825965
 0.859649  0.740351  0.741821  0.815439  0.825965
 0.860351  0.743158  0.739687  0.81614   0.82386
 0.860351  0.740351  0.743243  0.81614   0.820351
 0.863158  0.740351  0.745377  0.818246  0.818246
 0.863158  0.741754  0.746088  0.820351  0.818947
 0.864561  0.742456  0.747511  0.821053  0.819649
 0.864561  0.741754  0.751067  0.820351  0.818947
 ⋮                                       
 0.866667  0.745965  0.756757  0.816842  0.815439
 0.865965  0.745263  0.756046  0.817544  0.81614
 0.865965  0.745263  0.756046  0.817544  0.815439
 0.866667  0.745263  0.756046  0.818246  0.815439
 0.867368  0.745965  0.756757  0.817544  0.81614
 0.868772  0.746667  0.757468  0.81614

In [54]:
lambda_idx = argmax(mean(scores; dims=2)[:])
lambda = lambdas[lambda_idx]

1.2589254117941663e-9

In [55]:
U = ParProx.LogisticUpdate(; maxiter=30000, step=20, tol=1e-5, verbose=true)
V = ParProx.LogisticVariables{Float64}(adapt(A{T}, X), 
    adapt(A{T}, X_unpen), adapt(A{Int}, y), lambda, group_to_variables)
@time ParProx.fit!(U, V)

20	(-0.6882550177051432, Inf, 21956.0)
40	(-0.6837195236951092, 0.0026937348805460793, 21956.0)
60	(-0.6795117856550147, 0.0025053340357796446, 21965.0)
80	(-0.6756051154510823, 0.0023314981363498225, 21965.0)
100	(-0.6719749218217416, 0.0021712010042503073, 21965.0)
120	(-0.6685985792479503, 0.002023460055511381, 21965.0)
140	(-0.6654552969825818, 0.00188734111991083, 21965.0)
160	(-0.6625259907792304, 0.0017619611480350081, 21965.0)
180	(-0.6597931556525487, 0.0016464913819982819, 21965.0)
200	(-0.6572407470450712, 0.0015401555941878505, 21965.0)
220	(-0.6548540665157275, 0.0014422302108903038, 21965.0)
240	(-0.652619654399164, 0.0013520425650364518, 21965.0)
260	(-0.6505251896861826, 0.001268968644689206, 21965.0)
280	(-0.6485593943226992, 0.0011924322352310746, 21965.0)
300	(-0.6467119433282783, 0.0011219029544941822, 21965.0)
320	(-0.6449733910173964, 0.0010568878015751134, 21965.0)
340	(-0.6433350875183513, 0.000996938184724751, 21965.0)
360	(-0.6417891146766035, 0.00094163911060

2920	(-0.5835777545108944, 0.00016988954482147258, 21965.0)
2940	(-0.5833095708293466, 0.00016938170935663544, 21965.0)
2960	(-0.5830422308916687, 0.0001688773252292393, 21965.0)
2980	(-0.582775729952683, 0.00016837567947399116, 21965.0)
3000	(-0.5825100597336648, 0.00016787900802536174, 21965.0)
3020	(-0.5822452141572677, 0.00016738592351383174, 21965.0)
3040	(-0.5819811872417168, 0.00016689636873071077, 21965.0)
3060	(-0.5817179733777047, 0.00016641011131079473, 21965.0)
3080	(-0.5814555671409971, 0.00016592703719268994, 21965.0)
3100	(-0.5811939632913136, 0.0001654470329111343, 21965.0)
3120	(-0.5809331564401403, 0.00016497019504639656, 21965.0)
3140	(-0.5806731396585939, 0.00016449750111055417, 21965.0)
3160	(-0.5804139074755902, 0.00016402803200953563, 21965.0)
3180	(-0.5801554547311777, 0.00016356159366387482, 21965.0)
3200	(-0.5798977779690885, 0.0001630971102576303, 21965.0)
3220	(-0.5796408700486428, 0.00016263691660353528, 21965.0)
3240	(-0.5793847257453805, 0.000162179802733

5840	(-0.5511778981301063, 0.00012014039275206514, 21965.0)
5860	(-0.550991918136174, 0.0001199103565644753, 21965.0)
5880	(-0.5508063156955564, 0.00011968125144909525, 21965.0)
5900	(-0.5506210891464345, 0.00011945313424303965, 21965.0)
5920	(-0.5504362370964513, 0.00011922583177582032, 21965.0)
5940	(-0.5502517574686864, 0.00011899978624513, 21965.0)
5960	(-0.5500676486137245, 0.00011877472259133318, 21965.0)
5980	(-0.54988390889297, 0.00011855063447023673, 21965.0)
6000	(-0.5497005367349536, 0.00011832747919336272, 21965.0)
6020	(-0.5495175305195384, 0.00011810528878221566, 21965.0)
6040	(-0.549334888590231, 0.00011788408732832546, 21965.0)
6060	(-0.5491526093507461, 0.00011766383659347061, 21965.0)
6080	(-0.5489706912151561, 0.00011744453050126688, 21965.0)
6100	(-0.5487891326078018, 0.00011722616302746518, 21965.0)
6120	(-0.5486079327241218, 0.00011700823678546843, 21965.0)
6140	(-0.5484270895021864, 0.0001167915642663456, 21965.0)
6160	(-0.548246601143041, 0.00011657597634134199,

8760	(-0.5273279101029963, 9.466725034345378e-5, 21965.0)
8780	(-0.5271835368523136, 9.45356253514206e-5, 21965.0)
8800	(-0.5270393775555097, 9.4404439677687e-5, 21965.0)
8820	(-0.5268954314872089, 9.427369113326294e-5, 21965.0)
8840	(-0.5267516979256949, 9.414337754414374e-5, 21965.0)
8860	(-0.5266081761528866, 9.401349675067098e-5, 21965.0)
8880	(-0.5264648654543149, 9.388404660660148e-5, 21965.0)
8900	(-0.5263217651190975, 9.375502498079494e-5, 21965.0)
8920	(-0.5261788744399157, 9.362642975526455e-5, 21965.0)
8940	(-0.5260361927129913, 9.349825882620972e-5, 21965.0)
8960	(-0.525893719238062, 9.337051010366662e-5, 21965.0)
8980	(-0.5257514533183596, 9.324318151101295e-5, 21965.0)
9000	(-0.5256093942605865, 9.31162709849091e-5, 21965.0)
9020	(-0.5254675413748927, 9.298977647596682e-5, 21965.0)
9040	(-0.5253258939748531, 9.286369594796264e-5, 21965.0)
9060	(-0.5251844513774457, 9.27380273773417e-5, 21965.0)
9080	(-0.5250432129760818, 9.261272084763226e-5, 21965.0)
9100	(-0.52490217813

11660	(-0.5083211397180084, 7.928362767826792e-5, 21958.0)
11680	(-0.5082016926006147, 7.919837113283681e-5, 21958.0)
11700	(-0.5080823831471087, 7.911335271812741e-5, 21958.0)
11720	(-0.5079632109827995, 7.902856213021562e-5, 21958.0)
11740	(-0.5078441759940433, 7.894382632590818e-5, 21958.0)
11760	(-0.5077252777349746, 7.885936571108187e-5, 21958.0)
11780	(-0.5076065162159041, 7.877487779006966e-5, 21958.0)
11800	(-0.5074878908861946, 7.869073471614122e-5, 21958.0)
11820	(-0.5073694009989002, 7.860706686492304e-5, 21958.0)
11840	(-0.5072510461883886, 7.852362140393998e-5, 21958.0)
11860	(-0.5071328260905135, 7.844039744112057e-5, 21958.0)
11880	(-0.5070147403426059, 7.835739408943606e-5, 21958.0)
11900	(-0.5068967885834661, 7.827461046660982e-5, 21958.0)
11920	(-0.5067789704533558, 7.8192045695195e-5, 21958.0)
11940	(-0.5066612855939906, 7.810969890213645e-5, 21958.0)
11960	(-0.5065437336666447, 7.802755719527701e-5, 21958.0)
11980	(-0.5064263143099828, 7.794563567204351e-5, 21958.0)

...

17540	(-0.4779228077316957, 6.125798844165205e-5, 21958.0)
17560	(-0.4778323439100004, 6.121385965607016e-5, 21958.0)
17580	(-0.4777419507107437, 6.116981331768554e-5, 21958.0)
17600	(-0.4776516280001875, 6.11258491816784e-5, 21958.0)
17620	(-0.4775613756449938, 6.108196700412743e-5, 21958.0)
17640	(-0.47747119351222356, 6.103816654174744e-5, 21958.0)
17660	(-0.47738108146933445, 6.099444755275395e-5, 21958.0)
17680	(-0.47729103938418005, 6.09508097956992e-5, 21958.0)
17700	(-0.47720106712500787, 6.0907253030411976e-5, 21958.0)
17720	(-0.47711116456045766, 6.0863777017735266e-5, 21958.0)
17740	(-0.4770213315595604, 6.0820381519075837e-5, 21958.0)
17760	(-0.4769315679917365, 6.077706629693096e-5, 21958.0)
17780	(-0.47684187372679426, 6.0733831114851416e-5, 21958.0)
17800	(-0.4767522486349284, 6.0690675737066234e-5, 21958.0)
17820	(-0.4766626925867185, 6.064759992885911e-5, 21958.0)
17840	(-0.4765732054531277, 6.060460345638106e-5, 21958.0)
17860	(-0.4764837871055009, 6.056168608672619e-

20440	(-0.4654829764964606, 5.56102494841876e-5, 21950.0)
20460	(-0.4654015354517568, 5.557592423206029e-5, 21950.0)
20480	(-0.4653201491470003, 5.554165402273076e-5, 21950.0)
20500	(-0.465238817493139, 5.550743871259545e-5, 21950.0)
20520	(-0.465157540401354, 5.547327815870935e-5, 21950.0)
20540	(-0.4650763177830596, 5.54391722182179e-5, 21950.0)
20560	(-0.4649951495499017, 5.540512074926663e-5, 21950.0)
20580	(-0.4649140356137576, 5.537112361005419e-5, 21950.0)
20600	(-0.46483297588673467, 5.533718065970415e-5, 21950.0)
20620	(-0.46475197028117016, 5.530329175728002e-5, 21950.0)
20640	(-0.4646710187096296, 5.52694567629603e-5, 21950.0)
20660	(-0.46459012108490694, 5.523567553682605e-5, 21950.0)
20680	(-0.46450927732002284, 5.520194793988439e-5, 21950.0)
20700	(-0.46442848732822456, 5.516827383334874e-5, 21950.0)
20720	(-0.4643477510229848, 5.5134653079055954e-5, 21950.0)
20740	(-0.4642670683180013, 5.5101085539125425e-5, 21950.0)
20760	(-0.4641864391271952, 5.506757107664179e-5, 2195

23340	(-0.4542049987435585, 5.1143648422034065e-5, 21958.0)
23360	(-0.45413066936697205, 5.111602289416411e-5, 21958.0)
23380	(-0.45405638398203446, 5.108837989772679e-5, 21958.0)
23400	(-0.4539821425118187, 5.106078544233714e-5, 21958.0)
23420	(-0.4539079448877122, 5.103323382156578e-5, 21958.0)
23440	(-0.4538337909875262, 5.1005761900494156e-5, 21958.0)
23460	(-0.4537596807493748, 5.0978328215294656e-5, 21958.0)
23480	(-0.45368561411151537, 5.095093267789185e-5, 21958.0)
23500	(-0.4536115910123483, 5.0923575200366755e-5, 21958.0)
23520	(-0.453537611390416, 5.089625569549163e-5, 21958.0)
23540	(-0.4534636751844035, 5.086897407539346e-5, 21958.0)
23560	(-0.45338978233315785, 5.084173023910185e-5, 21957.0)
23580	(-0.45331593277605925, 5.0814523830025084e-5, 21957.0)
23600	(-0.45324212645177014, 5.078735535234612e-5, 21957.0)
23620	(-0.4531683632995394, 5.0760224412845825e-5, 21957.0)
23640	(-0.45309464325875515, 5.073313092596456e-5, 21957.0)
23660	(-0.4530209662689452, 5.07060748057272

26260	(-0.44378876862288275, 4.747460342323633e-5, 21957.0)
26280	(-0.44372026219607075, 4.7451316301256254e-5, 21957.0)
26300	(-0.4436517921388472, 4.742837406941762e-5, 21957.0)
26320	(-0.4435833581775264, 4.740561806363236e-5, 21957.0)
26340	(-0.44351496026692416, 4.738289001841065e-5, 21957.0)
26360	(-0.44344659836194983, 4.7360189876032026e-5, 21957.0)
26380	(-0.4433782724176062, 4.733751757894731e-5, 21957.0)
26400	(-0.443309982388989, 4.731487306985563e-5, 21957.0)
26420	(-0.44324172823128655, 4.729225629174294e-5, 21957.0)
26440	(-0.44317350997654653, 4.726961399196156e-5, 21957.0)
26460	(-0.4431053275364999, 4.724702954498204e-5, 21957.0)
26480	(-0.4430371808337499, 4.7224495428899884e-5, 21957.0)
26500	(-0.442969069823852, 4.720198881755115e-5, 21957.0)
26520	(-0.4429009944624528, 4.7179509654818934e-5, 21957.0)
26540	(-0.44283295470529066, 4.715705788410422e-5, 21957.0)
26560	(-0.4427649505081939, 4.713463345001879e-5, 21957.0)
26580	(-0.4426969818270822, 4.7112236296230737e

29100	(-0.43440303637545974, 4.449023949763792e-5, 21963.0)
29120	(-0.4343392500157672, 4.447090163074213e-5, 21963.0)
29140	(-0.43427549419669564, 4.445158501940358e-5, 21963.0)
29160	(-0.43421176888399843, 4.4432289624008896e-5, 21963.0)
29180	(-0.43414807404349265, 4.441301540516344e-5, 21963.0)
29200	(-0.43408440964105943, 4.4393762323342886e-5, 21963.0)
29220	(-0.4340207756426433, 4.437453033943524e-5, 21963.0)
29240	(-0.43395717201425243, 4.435531941412149e-5, 21963.0)
29260	(-0.43389359872195826, 4.433612950837895e-5, 21963.0)
29280	(-0.4338300557318952, 4.431696058332639e-5, 21963.0)
29300	(-0.433766543010261, 4.4297812599836974e-5, 21963.0)
29320	(-0.43370306052331586, 4.427868551942877e-5, 21963.0)
29340	(-0.43363960823738296, 4.425957930314199e-5, 21963.0)
29360	(-0.43357618611884774, 4.42404939125844e-5, 21963.0)
29380	(-0.43351279413415833, 4.4221429309040854e-5, 21963.0)
29400	(-0.4334494322498245, 4.4202385454480096e-5, 21963.0)
29420	(-0.4333861004324187, 4.418336231019

In [56]:
β_orig = vcat(grpmat * collect(V.β[1:end-2]), collect(V.β)[end-1:end]);

In [57]:
for (v, β) in zip(variable_names_replicated[V.β .!= 0], V.β[V.β .!= 0])
    println("$v\t$β")
end

KEGG:00280: Valine, leucine and isoleucine degradation	ABAT	4-aminobutyrate aminotransferase	0.000832354413401785
KEGG:00280: Valine, leucine and isoleucine degradation	AUH	AU RNA binding protein/enoyl-CoA hydratase	-0.0014593426888791473
KEGG:00280: Valine, leucine and isoleucine degradation	HMGCL	3-hydroxymethyl-3-methylglutaryl-CoA lyase	-0.0010615725170478396
KEGG:00280: Valine, leucine and isoleucine degradation	IVD	isovaleryl-CoA dehydrogenase	-0.0009348559171503038
KEGG:00280: Valine, leucine and isoleucine degradation	MCCC1	methylcrotonoyl-CoA carboxylase 1 (alpha)	8.306605556755163e-5
KEGG:00280: Valine, leucine and isoleucine degradation	MCCC2	methylcrotonoyl-CoA carboxylase 2 (beta)	-0.0007834944198441148
KEGG:00280: Valine, leucine and isoleucine degradation	MUT	methylmalonyl CoA mutase	0.002754888016414543
KEGG:04727: GABAergic synapse	ABAT	4-aminobutyrate aminotransferase	0.0008232735329612726
KEGG:04727: GABAergic synapse	GABRA4	gamma-aminobutyric acid (GABA) A receptor,

KEGG:00650: Butanoate metabolism	ABAT	4-aminobutyrate aminotransferase	0.0008342785686320108
KEGG:00650: Butanoate metabolism	ALDH5A1	aldehyde dehydrogenase 5 family, member A1	0.0023314907318213746
KEGG:00650: Butanoate metabolism	BDH1	3-hydroxybutyrate dehydrogenase, type 1	-0.00020734363851731568
KEGG:00650: Butanoate metabolism	HMGCL	3-hydroxymethyl-3-methylglutaryl-CoA lyase	-0.0010643922027909467
GO:0001666: response to hypoxia	ABAT	4-aminobutyrate aminotransferase	0.0008334434534379149
GO:0001666: response to hypoxia	ADM	adrenomedullin	0.0022388125598973465
GO:0001666: response to hypoxia	AGER	advanced glycosylation end product-specific receptor	-0.00017426454237396923
GO:0001666: response to hypoxia	BNIP3	BCL2/adenovirus E1B 19kDa interacting protein 3	0.0029141508436547
GO:0001666: response to hypoxia	CAV1	caveolin 1, caveolae protein, 22kDa	-0.0005354719528487099
GO:0001666: response to hypoxia	CCL2	chemokine (C-C motif) ligand 2	0.0008172213365582937
GO:0001666: response to 

GO:0035094: response to nicotine	BCL2	B-cell CLL/lymphoma 2	-5.086636827417194e-6
GO:0035094: response to nicotine	CHRNB2	cholinergic receptor, nicotinic, beta 2 (neuronal)	-0.0007583278132899442
GO:0035094: response to nicotine	DRD2	dopamine receptor D2	-0.0002300111499356166
GO:0035094: response to nicotine	GPX1	glutathione peroxidase 1	0.00038673054565649677
GO:0035094: response to nicotine	HDAC2	histone deacetylase 2	-0.0011885414992245907
GO:0035094: response to nicotine	HMOX1	heme oxygenase (decycling) 1	-0.0002107114039207383
GO:0035094: response to nicotine	PDX1	pancreatic and duodenal homeobox 1	0.0002469304872419898
GO:0035094: response to nicotine	STAR	steroidogenic acute regulatory protein	0.0008692233019067948
GO:0042135: neurotransmitter catabolic process	ABAT	4-aminobutyrate aminotransferase	0.0008384285830425876
GO:0042135: neurotransmitter catabolic process	ALDH5A1	aldehyde dehydrogenase 5 family, member A1	0.0023452776173856042
GO:0042493: response to drug	ABAT	4-amin

GO:0006355: regulation of transcription, DNA-templated	CEBPB	CCAAT/enhancer binding protein (C/EBP), beta	-0.00027454497597901374
GO:0006355: regulation of transcription, DNA-templated	CELSR2	cadherin, EGF LAG seven-pass G-type receptor 2	-0.0011360981432418868
GO:0006355: regulation of transcription, DNA-templated	CHAF1B	chromatin assembly factor 1, subunit B (p60)	0.006292511066352086
GO:0006355: regulation of transcription, DNA-templated	CHD3	chromodomain helicase DNA binding protein 3	0.00011917125375918129
GO:0006355: regulation of transcription, DNA-templated	CLOCK	clock circadian regulator	0.00020035738705752543
GO:0006355: regulation of transcription, DNA-templated	DAXX	death-domain associated protein	0.0008926960114535765
GO:0006355: regulation of transcription, DNA-templated	DLX6	distal-less homeobox 6	-0.00018007477071055284
GO:0006355: regulation of transcription, DNA-templated	E2F1	E2F transcription factor 1	0.002100173092073431
GO:0006355: regulation of transcription, DNA

GO:0032367: intracellular cholesterol transport	ABCG1	ATP-binding cassette, sub-family G (WHITE), member 1	-0.0023414183670599836
GO:0032367: intracellular cholesterol transport	STAR	steroidogenic acute regulatory protein	0.0008955133587820496
GO:0032367: intracellular cholesterol transport	VPS4A	vacuolar protein sorting 4 homolog A (S. cerevisiae)	-0.0006857834708294199
GO:0033344: cholesterol efflux	ABCG1	ATP-binding cassette, sub-family G (WHITE), member 1	-0.0023351541238514502
GO:0033344: cholesterol efflux	CAV1	caveolin 1, caveolae protein, 22kDa	-0.0005351373609785365
GO:0033344: cholesterol efflux	SOAT1	sterol O-acyltransferase 1	-0.00040815997631377283
GO:0033700: phospholipid efflux	ABCG1	ATP-binding cassette, sub-family G (WHITE), member 1	-0.0023606226924747856
GO:0033993: response to lipid	ABCG1	ATP-binding cassette, sub-family G (WHITE), member 1	-0.0023480539884438473
GO:0033993: response to lipid	GATA2	GATA binding protein 2	-0.0004546861240076836
GO:0033993: response t

...

GO:0019722: calcium-mediated signaling	AGTR1	angiotensin II receptor, type 1	0.0003470865204972392
GO:0019722: calcium-mediated signaling	CXCR4	chemokine (C-X-C motif) receptor 4	0.0007919562854707342
GO:0019722: calcium-mediated signaling	SPHK1	sphingosine kinase 1	0.0018046616123086462
GO:0032430: positive regulation of phospholipase A2 activity	AGTR1	angiotensin II receptor, type 1	0.0003408727207204732
GO:0038166: angiotensin-activated signaling pathway	AGTR1	angiotensin II receptor, type 1	0.0003348582837193238
GO:0038166: angiotensin-activated signaling pathway	MAS1	MAS1 proto-oncogene, G protein-coupled receptor	0.00030176515636567427
GO:0042312: regulation of vasodilation	AGTR1	angiotensin II receptor, type 1	0.0003408727207204732
GO:0050727: regulation of inflammatory response	AGTR1	angiotensin II receptor, type 1	0.00034965335038647944
GO:0050727: regulation of inflammatory response	BRD4	bromodomain containing 4	0.0005486325697231975
GO:0050727: regulation of inflammatory res

GO:0006915: apoptotic process	TPX2	TPX2, microtubule-associated	0.0013408903692152202
GO:0006915: apoptotic process	TRAIP	TRAF interacting protein	0.0010220359191476969
GO:0031398: positive regulation of protein ubiquitination	AIMP2	aminoacyl tRNA synthetase complex-interacting multifunctional protein 2	0.001201006959874168
GO:0031398: positive regulation of protein ubiquitination	ANGPT1	angiopoietin 1	0.0008913658074095579
GO:0031398: positive regulation of protein ubiquitination	AXIN1	axin 1	-0.0007213068450650998
GO:0031398: positive regulation of protein ubiquitination	CAV1	caveolin 1, caveolae protein, 22kDa	-0.0005316868040034787
GO:0031398: positive regulation of protein ubiquitination	CDK5RAP3	CDK5 regulatory subunit associated protein 3	0.0011275586233072874
GO:0031398: positive regulation of protein ubiquitination	FGFR3	fibroblast growth factor receptor 3	0.00021868572785721453
GO:0031398: positive regulation of protein ubiquitination	RCHY1	ring finger and CHY zinc finger dom

GO:0051260: protein homooligomerization	KCNV2	potassium channel, subfamily V, member 2	0.0009177516646332515
GO:0051260: protein homooligomerization	RAD51	RAD51 recombinase	-0.0018429989679449607
GO:0051260: protein homooligomerization	SLC1A1	solute carrier family 1 (neuronal/epithelial high affinity glutamate transporter, system Xag), member 1	-0.0008918154514029547
GO:0051260: protein homooligomerization	SYT1	synaptotagmin I	-0.0014261697188205772
GO:0071395: cellular response to jasmonic acid stimulus	AKR1C1	aldo-keto reductase family 1, member C1	-0.0014835116889152422
KEGG:04510: Focal adhesion	AKT1	v-akt murine thymoma viral oncogene homolog 1	0.0007311119635755518
KEGG:04510: Focal adhesion	AKT2	v-akt murine thymoma viral oncogene homolog 2	-0.0006259514437814401
KEGG:04510: Focal adhesion	BCL2	B-cell CLL/lymphoma 2	-5.08703490194434e-6
KEGG:04510: Focal adhesion	CAV1	caveolin 1, caveolae protein, 22kDa	-0.0005354547470216653
KEGG:04510: Focal adhesion	CCND1	cyclin D1	-0.0026352

Excessive output truncated after 524364 bytes.

KEGG:05169: Epstein-Barr virus infection	IRAK1	interleukin-1 receptor-associated kinase 1	-0.0016002937427628534
KEGG:05169: Epstein-Barr virus infection	LYN	LYN proto-oncogene, Src family tyrosine kinase	-0.0014136847532094162
KEGG:05169: Epstein-Barr virus infection	MYC	v-myc avian myelocytomatosis viral oncogene homolog	-0.001132511741412116
KEGG:05169: Epstein-Barr virus infection	NEDD4	neural precursor cell expressed, developmentally down-regulated 4, E3 ubiquitin protein ligase	0.002253522858241044
KEGG:05169: Epstein-Barr virus infection	PIK3CD	phosphatidylinositol-4,5-bisphosphate 3-kinase, catalytic subunit delta	-0.0007767642901220775
KEGG:05169: Epstein-Barr virus infection	POLR2D	polymerase (RNA) II (DNA directed) polypeptide D	0.0018450093234024662
KEGG:05169: Epstein-Barr virus infection	PRKX	protein kinase, X-linked	0.0003903024168112265
KEGG:05169: Epstein-Barr virus infection	PSMD11	proteasome (prosome, macropain) 26S subunit, non-ATPase, 11	-0.001091548683016147
KEGG:

In [58]:
for (v, β) in zip(variable_names[β_orig .!= 0], β_orig[β_orig .!= 0])
    println("$v\t$β")
end

RFC2	0.01782837035434921
PAX8	0.03684757874519626
THRA	0.02806710700734747
EPHB3	0.019420774902517746
CFL1	-0.030460550648403986
YY1	0.013402782876598915
ZPR1	-0.004170459413053395
RHOA	0.04322099220670631
GUK1	0.05284161532839052
DSP	-0.0008924333259685115
RAD21	-0.001631468584420958
SF3B2	0.004117778492081898
NDRG1	-0.006543628992920167
CD63	-0.01588590376579995
DNAJB1	-0.002074023570791528
XBP1	0.021161233616516416
SPTBN1	0.03684346496032825
HMGB1	-0.030756596050312522
GPX1	0.021234007378015077
NUMA1	0.02134145223823748
ARL6IP5	0.0010924161799196037
STMN1	0.00724792250908532
ODC1	0.00984689697218672
DAZAP2	-0.0007855012627521644
TMBIM6	-0.002281876930489588
GSTP1	0.008155197247241446
ZNF207	-0.00272055661792677
DHCR24	-0.03582951324451189
DEK	0.017525131494012178
HIF1A	-0.04980124402515673
SNX17	0.011478189129344105
CD9	-0.038892803436755025
JUP	-0.038198768007394066
TGM2	-0.001727118661130326
MMP2	-0.025981932630592196
THBS1	-0.08446415723750327
TUFM	-0.006526015058534711
POLD2	0.0

ATF5	-0.0438895183388974
RAD51	-0.05959231669664289
CCNE2	0.018853635423271606
ORM1	-0.0027846959939131547
CENPE	0.03231574820149773
PSPH	-0.021255067676917743
KIT	0.023303054440520186
AUH	-0.00874276593188335
PRIM1	0.007640062297202153
SLC22A5	-0.02409020140972214
SERPINF2	0.014325953337234942
CRIP1	0.01716096384814977
ORC1	-0.0058874352807807015
MOK	0.005371912559628551
PEX11A	0.0063813019014951405
GLI3	-0.14256324992470196
KAL1	-0.0011795132247511
GALK2	0.00012695444364892622
ESR1	-0.04057493084726801
AREG	0.05169677369466026
GPSM2	0.007115593911870687
CXCL13	0.0783715582690652
EGR2	0.03901184423000849
PER2	0.03325074136646313
PGC	0.005304266152110367
LRP8	-0.0204630241206002
TFAP2C	0.020189519687844056
CDC14A	0.0023230844116615816
IGFBP1	0.003091742143176988
SLC15A2	-0.0010016510545315755
S100A1	0.007943911280445219
DCT	-0.03064933540650469
BARD1	0.020946437515230745
AGTR1	0.012107886892131919
INPP4B	0.0043235597350753455
PDZK1	-0.0015817353024116046
CHEK1	0.01974575498226775
MLLT1

CDC34	-0.002165270035415801
ZNF609	-0.0020932604936431992
METAP1	-0.00802137483893706
SLC25A44	0.0003453198541108316
JMJD6	0.07225247109460836
C16orf45	0.0015129853607803165
RCHY1	-0.012047180085251727
BBS4	-0.02449128618535784
TCF7L2	0.022737387684181258
FAM171A1	-0.0008474461993496484
OBSL1	0.0004996753392981065
NCAPD3	-0.00629194532409631
CBS	0.027408147568601854
CKAP5	-0.007844218411740492
POLD3	0.04009422205891675
AXIN1	-0.04008779072142446
DNAJC16	-0.00010774179655897069
SMC5	-0.008404355246720133
NCAPH	0.0005077986713116512
KANK1	0.03570418486402973
NEDD4	0.08611271769809645
LOC100272216	-0.000490130399845661
DNAJC2	-0.00785480274112632
HOXA10	-0.03585348281798796
ROBO1	-0.03381507510547673
TNNT1	0.00236167027647398
FOXC1	-0.0010032433396977875
YAP1	-0.005210933274387358
ATAD2B	-0.0012403210541986128
SPDEF	0.031499791704416746
SOSTDC1	-0.012182241980776628
CLCN2	0.008450306755612699
CCNE1	0.013357563921641032
DNA2	-0.0005924515358255859
DLX5	-0.06904625505431027
ELOVL2	0.0060805